<a href="https://colab.research.google.com/github/mejriimariemm-12/veristream-x/blob/main/detection%2Bintegration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from io import StringIO
import random
import re

print("🏥 Création Dataset Misinformation Médicale - 5000 Exemples\n")

# ========== ÉTAPE 1: TÉLÉCHARGER DATASET RÉEL ==========
print("="*70)
print("📥 TÉLÉCHARGEMENT DATASET RÉEL")
print("="*70)

def download_kaggle_covid():
    """Télécharge le seul dataset qui fonctionne"""
    print("⬇️  COVID_Fake_Kaggle... ", end="")
    try:
        url = "https://raw.githubusercontent.com/susanli2016/NLP-with-Python/master/data/corona_fake.csv"
        headers = {'User-Agent': 'Mozilla/5.0'}
        r = requests.get(url, headers=headers, timeout=30)
        r.raise_for_status()

        df = pd.read_csv(StringIO(r.text), encoding='utf-8', on_bad_lines='skip')
        print(f"✅ {len(df):,} lignes téléchargées")
        return df
    except Exception as e:
        print(f"❌ {e}")
        return None

def normalize_dataset(df):
    """Normalise le dataset"""
    print("   🔧 Normalisation... ", end="")

    # Détecter colonnes
    text_col = None
    label_col = None

    for col in df.columns:
        if 'title' in col.lower() or 'text' in col.lower():
            text_col = col
        if 'label' in col.lower():
            label_col = col

    if not text_col or not label_col:
        print("❌ Colonnes non trouvées")
        return None

    df_clean = df[[text_col, label_col]].copy()
    df_clean.columns = ['text', 'label']

    # Nettoyage texte
    df_clean = df_clean.dropna()
    df_clean['text'] = df_clean['text'].astype(str).str.strip()
    df_clean = df_clean[df_clean['text'].str.len() > 30]

    # Convertir labels avec gestion des NaN
    if df_clean['label'].dtype == 'object':
        label_map = {'fake': 1, 'real': 0, 'false': 1, 'true': 0}
        df_clean['label'] = df_clean['label'].str.lower().str.strip().map(label_map)

    # Supprimer les NaN avant conversion en int
    df_clean = df_clean.dropna(subset=['label'])

    # Convertir en numérique puis int
    df_clean['label'] = pd.to_numeric(df_clean['label'], errors='coerce')
    df_clean = df_clean.dropna(subset=['label'])
    df_clean['label'] = df_clean['label'].astype(int)

    # Garder seulement 0 et 1
    df_clean = df_clean[df_clean['label'].isin([0, 1])]
    df_clean = df_clean.drop_duplicates(subset=['text'])

    print(f"✅ {len(df_clean)} exemples normalisés")
    return df_clean

# Téléchargement
df_real = download_kaggle_covid()
if df_real is not None:
    df_real = normalize_dataset(df_real)
    df_real['source'] = 'kaggle_covid'
else:
    df_real = pd.DataFrame(columns=['text', 'label', 'source'])

print(f"\n📊 Dataset réel: {len(df_real)} exemples")

# ========== ÉTAPE 2: DATA AUGMENTATION ==========
print(f"\n{'='*70}")
print("🔄 DATA AUGMENTATION - GÉNÉRATION INTELLIGENTE")
print("="*70)

# Templates pour variation de style
PARAPHRASE_TEMPLATES = {
    'formal': [
        "{claim}",
        "Des études suggèrent que {claim}",
        "Il a été démontré que {claim}",
        "Les recherches indiquent que {claim}",
        "Selon les experts, {claim}",
        "D'après les données médicales, {claim}",
        "Les professionnels de santé affirment que {claim}"
    ],
    'informal': [
        "{claim}",
        "Tu savais que {claim}?",
        "Incroyable mais vrai: {claim}",
        "J'ai entendu dire que {claim}",
        "Apparemment {claim}",
        "Il paraît que {claim}",
        "On m'a dit que {claim}"
    ],
    'question': [
        "{claim}?",
        "Est-ce que {claim}?",
        "Est-il vrai que {claim}?",
        "Peut-on dire que {claim}?",
        "Savez-vous que {claim}?",
        "Avez-vous entendu que {claim}?"
    ],
    'news': [
        "BREAKING: {claim}",
        "Alerte santé: {claim}",
        "Info exclusive: {claim}",
        "Dernière minute: {claim}",
        "Révélation choc: {claim}",
        "Nouvelle étude: {claim}"
    ],
    'social': [
        "{claim} #santé #medical",
        "{claim} 🔬💉",
        "RT: {claim}",
        "THREAD: {claim}",
        "À partager: {claim}",
        "Important: {claim}"
    ]
}

# FAKE NEWS MÉDICALES (100 variations)
FAKE_CLAIMS = [
    # COVID-19 (30)
    "le vaccin COVID contient des micropuces de traçage",
    "la 5G a causé la pandémie de coronavirus",
    "boire de l'eau chaude tue le virus COVID-19",
    "les masques causent une intoxication au CO2",
    "le coronavirus a été créé en laboratoire",
    "la vitamine C guérit complètement le COVID",
    "les tests PCR implantent des nano-robots",
    "le vaccin COVID modifie l'ADN humain",
    "l'hydroxychloroquine guérit 100% des cas",
    "les enfants ne peuvent pas attraper le COVID",
    "le vaccin cause l'infertilité permanente",
    "les masques N95 provoquent des maladies",
    "le zinc prévient complètement le COVID",
    "les vaccins contiennent des fœtus avortés",
    "l'immunité naturelle dure pour toujours",
    "le vaccin contient de la luciférase",
    "boire de l'alcool fort tue le coronavirus",
    "les tests COVID sont faux à 80%",
    "le virus a été breveté avant 2019",
    "les vaccins causent des caillots sanguins massifs",
    "l'ivermectine est meilleure que les vaccins",
    "les hôpitaux gonflent les chiffres de morts",
    "le masque endommage le cerveau par hypoxie",
    "la 5G active le virus dans le corps",
    "les vaccins contiennent du graphène magnétique",
    "le COVID disparaît avec l'été",
    "les vaccins causent des variants",
    "l'ail et le gingembre guérissent le COVID",
    "le virus ne survit pas sur les surfaces",
    "les asymptomatiques ne transmettent pas le virus",

    # Vaccins (25)
    "les vaccins causent l'autisme chez les enfants",
    "les vaccins contiennent du mercure toxique",
    "l'immunité naturelle est supérieure aux vaccins",
    "Big Pharma cache les dangers des vaccins",
    "les vaccins affaiblissent le système immunitaire",
    "les enfants non vaccinés sont plus sains",
    "les vaccins causent la mort subite du nourrisson",
    "l'aluminium des vaccins empoisonne le cerveau",
    "les maladies disparaissent sans vaccins",
    "les vaccins causent plus de maladies qu'ils préviennent",
    "le calendrier vaccinal surcharge l'immunité",
    "les vaccins contiennent des virus dangereux",
    "l'industrie invente des maladies pour vendre",
    "les vaccins causent des allergies",
    "les vaccins provoquent des troubles neurologiques",
    "les vaccins ne sont pas testés correctement",
    "les non-vaccinés vivent plus longtemps",
    "les vaccins causent le diabète",
    "les adjuvants vaccinaux sont toxiques",
    "la vaccination de masse est un génocide",
    "les vaccins contiennent des cellules animales",
    "l'immunité de groupe est un mythe",
    "les vaccins causent la sclérose en plaques",
    "la rougeole n'est pas dangereuse",
    "les vaccins causent l'asthme infantile",

    # Cancer (20)
    "le bicarbonate guérit tous les cancers",
    "les médecins cachent le remède contre le cancer",
    "le cancer est un champignon traitable",
    "les micro-ondes causent le cancer",
    "le jeûne guérit le cancer en 2 semaines",
    "la chimiothérapie tue plus qu'elle guérit",
    "le cannabis guérit tous les cancers",
    "les déodorants causent le cancer du sein",
    "le sucre nourrit directement le cancer",
    "les téléphones causent des tumeurs cérébrales",
    "le citron détruit les cellules cancéreuses",
    "la vitamine B17 guérit le cancer",
    "les mammographies causent le cancer",
    "le cancer est causé par un parasite",
    "l'eau alcaline prévient le cancer",
    "la chimiothérapie est un poison inutile",
    "les antioxydants guérissent le cancer",
    "le cancer disparaît avec un régime cru",
    "la médecine cache les vrais remèdes",
    "le curcuma guérit le cancer avancé",

    # Remèdes miracles (25)
    "l'eau de javel MMS guérit cancer et SIDA",
    "les huiles essentielles remplacent les médicaments",
    "l'homéopathie équivaut aux antibiotiques",
    "l'ail cru prévient toutes les infections",
    "le vinaigre de cidre guérit le diabète",
    "les cristaux guérissent par vibration",
    "l'urinothérapie soigne toutes maladies",
    "le magnétisme guérit les maladies chroniques",
    "le jeûne sec élimine toutes toxines",
    "l'argent colloïdal tue tous les virus",
    "les ondes scalaires guérissent tout",
    "l'eau structurée régénère les cellules",
    "le charbon actif détoxifie complètement",
    "les aimants guérissent l'arthrite",
    "le peroxyde d'hydrogène soigne le cancer",
    "les plantes guérissent mieux que les médicaments",
    "la thérapie par les couleurs guérit",
    "le jus de céleri guérit toutes maladies",
    "les champignons médicinaux remplacent la chimio",
    "l'acupuncture guérit sans médicaments",
    "les probiotiques guérissent la dépression",
    "le jeûne intermittent guérit le diabète",
    "l'huile de coco guérit Alzheimer",
    "le cuivre ionique tue tous les microbes",
    "la méditation guérit le cancer"
]

# VRAIES INFORMATIONS (100 variations)
REAL_CLAIMS = [
    # COVID-19 (30)
    "les vaccins COVID réduisent les hospitalisations",
    "les masques réduisent la transmission virale",
    "la distanciation sociale limite la propagation",
    "le lavage des mains prévient les infections",
    "les vaccins COVID ont été testés rigoureusement",
    "les effets secondaires vaccinaux sont rares",
    "la vaccination réduit le risque de COVID long",
    "les tests PCR détectent fiablement le virus",
    "la ventilation améliore la sécurité",
    "les variants peuvent échapper partiellement à l'immunité",
    "le COVID cause des complications respiratoires",
    "la vaccination protège les vulnérables",
    "les anticorps monoclonaux traitent les cas sévères",
    "l'isolement des cas réduit la transmission",
    "la vaccination COVID est sûre pendant la grossesse",
    "les masques N95 offrent une meilleure protection",
    "la réinfection COVID est possible",
    "les comorbidités augmentent le risque",
    "la vaccination réduit la charge virale",
    "le COVID affecte plusieurs organes",
    "les vaccins sont efficaces contre les formes graves",
    "le dépistage précoce améliore les résultats",
    "les mesures sanitaires ont sauvé des vies",
    "la vaccination collective protège les communautés",
    "les traitements COVID s'améliorent constamment",
    "le COVID peut causer des séquelles long terme",
    "les vaccins à ARNm sont sûrs et efficaces",
    "la surveillance épidémiologique est cruciale",
    "les rappels vaccinaux maintiennent l'immunité",
    "la recherche COVID avance rapidement",

    # Prévention (25)
    "l'exercice régulier améliore la santé cardiaque",
    "une alimentation équilibrée réduit les maladies",
    "le tabagisme cause le cancer du poumon",
    "le dépistage précoce améliore la survie",
    "la vaccination a éradiqué la variole",
    "l'hygiène des mains prévient les infections",
    "la protection solaire réduit le cancer cutané",
    "l'activité physique réduit le diabète",
    "le sommeil adéquat est essentiel",
    "la réduction du stress améliore la santé",
    "l'hydratation est importante pour le corps",
    "les examens réguliers détectent les problèmes",
    "la vaccination antigrippale réduit les complications",
    "l'allaitement renforce l'immunité du bébé",
    "la réduction de sel diminue l'hypertension",
    "l'arrêt du tabac améliore la santé",
    "la vaccination protège contre les maladies",
    "l'exercice prévient l'ostéoporose",
    "une bonne hygiène dentaire prévient les maladies",
    "la modération d'alcool protège le foie",
    "le contrôle du poids réduit les maladies",
    "la prévention est plus efficace que le traitement",
    "les fibres alimentaires améliorent la digestion",
    "la vitamine D est importante pour les os",
    "les omega-3 sont bénéfiques pour le cœur",

    # Traitements (25)
    "les antibiotiques ne traitent pas les virus",
    "la chimiothérapie traite efficacement certains cancers",
    "les vaccins ont sauvé des millions de vies",
    "l'insuline est vitale pour le diabète type 1",
    "les antirétroviraux permettent de vivre avec le VIH",
    "la radiothérapie cible précisément les tumeurs",
    "les antihypertenseurs préviennent les AVC",
    "les statines réduisent les maladies cardiaques",
    "l'immunothérapie montre des résultats prometteurs",
    "les anticoagulants préviennent les caillots",
    "les inhalateurs contrôlent l'asthme",
    "les antidépresseurs aident de nombreuses personnes",
    "la thérapie cognitivo-comportementale traite l'anxiété",
    "les vaccins réduisent la charge des maladies",
    "les traitements hormonaux sont efficaces",
    "la dialyse maintient la vie en insuffisance rénale",
    "les transplantations sauvent des vies",
    "les antibiotiques ont révolutionné la médecine",
    "la chirurgie peut guérir certains cancers",
    "les médicaments sont testés scientifiquement",
    "la médecine moderne prolonge la vie",
    "les traitements s'améliorent constamment",
    "la recherche médicale sauve des vies",
    "les essais cliniques sont essentiels",
    "la médecine factuelle guide les traitements",

    # Science médicale (20)
    "les essais randomisés sont l'étalon-or",
    "les médicaments passent par des phases de tests",
    "la médecine s'appuie sur des preuves",
    "les effets placebo influencent les résultats",
    "la résistance aux antibiotiques est un problème",
    "la recherche médicale est rigoureuse",
    "les pairs évaluent les publications scientifiques",
    "les études cliniques suivent des protocoles stricts",
    "la science médicale évolue constamment",
    "les données probantes guident les décisions",
    "la médecine basée sur les preuves est essentielle",
    "les biais doivent être contrôlés dans les études",
    "la reproductibilité valide les résultats",
    "les méta-analyses synthétisent les données",
    "l'éthique médicale protège les patients",
    "la transparence scientifique est cruciale",
    "les conflits d'intérêts doivent être déclarés",
    "la recherche fondamentale est essentielle",
    "la collaboration scientifique accélère les progrès",
    "le consensus médical émerge des preuves"
]

def augment_claims(claims, label, target_count):
    """Génère des variations de claims avec différents styles"""
    augmented = []
    styles = list(PARAPHRASE_TEMPLATES.keys())

    while len(augmented) < target_count:
        claim = random.choice(claims)
        style = random.choice(styles)
        template = random.choice(PARAPHRASE_TEMPLATES[style])

        # Formater le claim
        text = template.format(claim=claim)

        # Variations aléatoires
        if random.random() > 0.7:
            # Ajouter ponctuation
            if not text.endswith(('!', '?', '.')):
                text += random.choice(['.', '!', ''])

        if random.random() > 0.8:
            # Ajouter émojis
            emojis = ['💉', '🔬', '🏥', '⚕️', '🩺', '💊', '🧬']
            text += ' ' + random.choice(emojis)

        augmented.append({
            'text': text,
            'label': label,
            'source': 'augmented'
        })

    return augmented

# Calculer combien générer
total_real = len(df_real[df_real['label'] == 0]) if len(df_real) > 0 else 0
total_fake = len(df_real[df_real['label'] == 1]) if len(df_real) > 0 else 0

target_per_class = 2500
fake_needed = target_per_class - total_fake
real_needed = target_per_class - total_real

print(f"📊 Dataset réel:")
print(f"   • Fake: {total_fake}")
print(f"   • Real: {total_real}")
print(f"\n🎯 Génération nécessaire:")
print(f"   • Fake à générer: {fake_needed}")
print(f"   • Real à générer: {real_needed}")

# Générer les données augmentées
print(f"\n⚙️  Génération en cours...")
fake_augmented = augment_claims(FAKE_CLAIMS, 1, fake_needed)
real_augmented = augment_claims(REAL_CLAIMS, 0, real_needed)

df_fake_aug = pd.DataFrame(fake_augmented)
df_real_aug = pd.DataFrame(real_augmented)

print(f"   ✅ Fake générés: {len(df_fake_aug)}")
print(f"   ✅ Real générés: {len(df_real_aug)}")

# ========== ÉTAPE 3: FUSION ==========
print(f"\n{'='*70}")
print("🔗 FUSION FINALE")
print("="*70)

# Combiner tous les datasets
all_dfs = [df_real, df_fake_aug, df_real_aug]
df_final = pd.concat(all_dfs, ignore_index=True)

# Mélanger aléatoirement
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

# Limiter à exactement 5000
df_final = df_final.head(5000)

# Sauvegarde
output_file = "medical_final_isolated.csv"
df_final.to_csv(output_file, index=False, encoding='utf-8')

# ========== STATISTIQUES FINALES ==========
print(f"\n{'='*70}")
print(f"✨ DATASET FINAL CRÉÉ - 5000 EXEMPLES")
print(f"{'='*70}")

fake_final = (df_final['label'] == 1).sum()
real_final = (df_final['label'] == 0).sum()

print(f"📊 Distribution:")
print(f"   • Total: {len(df_final):,} exemples")
print(f"   • Misinformation (1): {fake_final:,} ({fake_final/len(df_final):.1%})")
print(f"   • Vraies infos (0):   {real_final:,} ({real_final/len(df_final):.1%})")

balance = abs(fake_final - real_final)
print(f"   • Balance: ±{balance} exemples ({balance/len(df_final)*100:.1f}%)")

if balance < 100:
    print(f"   ✅ PARFAITEMENT ÉQUILIBRÉ")
else:
    print(f"   ⚠️  Légèrement déséquilibré")

print(f"\n📈 Par source:")
for source, count in df_final['source'].value_counts().items():
    print(f"   • {source}: {count:,} ({count/len(df_final)*100:.1f}%)")

print(f"\n📁 Fichier sauvegardé: {output_file}")

# Exemples variés
print(f"\n{'='*70}")
print("📝 EXEMPLES GÉNÉRÉS")
print("="*70)

print(f"\n🚫 MISINFORMATION (échantillon):")
for i, text in enumerate(df_final[df_final['label']==1]['text'].sample(5), 1):
    print(f"   {i}. {text}")

print(f"\n✅ VRAIES INFORMATIONS (échantillon):")
for i, text in enumerate(df_final[df_final['label']==0]['text'].sample(5), 1):
    print(f"   {i}. {text}")

print(f"\n{'='*70}")
print(f"🎯 PRÊT POUR MACHINE LEARNING")
print(f"{'='*70}")
print(f"💡 Caractéristiques:")
print(f"   ✅ 5000 exemples au total")
print(f"   ✅ Balance 50/50 fake/real")
print(f"   ✅ Styles variés (formel, informel, questions, news, social)")
print(f"   ✅ Augmentation intelligente avec templates")
print(f"   ✅ Dataset réel + synthétique haute qualité")
print(f"\n🚀 Prochaine étape: Entraîner votre modèle de détection!")
print(f"{'='*70}")


In [ ]:
# ÉTAPE 1 - JUSTE CETTE LIGNE
df = pd.read_csv('medical_final_isolated.csv')
print(f"✅ Dataset chargé: {df.shape}")

In [ ]:
# CELLULE 3 - CHARGEMENT DES DONNÉES GÉNÉRÉES
print("📁 Chargement du nouveau dataset de qualité...")

# Maintenant on utilise le fichier qu'on vient de créer
df = pd.read_csv('medical_final_isolated.csv')

print(f"📊 Dataset de qualité: {df.shape}")
print(f"🎯 Distribution: {df['label'].value_counts()}")

# Les colonnes sont maintenant standardisées
texts = df['text'].fillna('').tolist()
labels = df['label'].values

print(f"✅ {len(texts)} textes uniques prêts pour l'analyse!")
print(f"🔤 Exemple: {texts[0][:100]}...")

In [ ]:
# CELLULE 3 - SPARK FEATURE ENGINEERING (CORRIGÉE)
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import *
from pyspark.ml import Pipeline  # ⬅️ AJOUT IMPORT MANQUANT

print("🔥 Initialisation Spark pour feature engineering...")

spark = SparkSession.builder \
    .appName("DL_Features") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

# Conversion en DataFrame Spark
df_spark = spark.createDataFrame(df)

# Pipeline de features avancées
tokenizer = Tokenizer(inputCol="text", outputCol="words")
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
hashing_tf = HashingTF(inputCol="filtered_words", outputCol="tf_features", numFeatures=300)
idf = IDF(inputCol="tf_features", outputCol="tfidf_features")
word2vec = Word2Vec(vectorSize=100, minCount=1, inputCol="filtered_words", outputCol="w2v_features")

pipeline = Pipeline(stages=[tokenizer, stopwords_remover, hashing_tf, idf, word2vec])
pipeline_model = pipeline.fit(df_spark)
df_features = pipeline_model.transform(df_spark)

print("✅ Features Spark générées!")
print(f"📊 Nombre de features créées: {df_features.count()} lignes")
df_features.select("text", "tfidf_features", "w2v_features").show(2, truncate=50)

In [ ]:
# CELLULE 4 - PRÉPARATION FEATURES POUR DL
print("🔄 Conversion des features Spark pour Deep Learning...")

import numpy as np

# Conversion en Pandas
df_final = df_features.toPandas()

# Extraction features Spark
spark_tfidf = np.array([f.toArray() for f in df_final['tfidf_features']])
spark_w2v = np.array([f.toArray() for f in df_final['w2v_features']])

# Combinaison features
spark_features = np.concatenate([spark_tfidf, spark_w2v], axis=1)

# Textes et labels
texts = df_final['text'].fillna('').tolist()
labels = df_final['label'].values

print(f"📊 Données finales: {len(texts)} textes, {spark_features.shape[1]} features")
print(f"🔢 Shape features: {spark_features.shape}")
print(f"🎯 Distribution labels: {np.unique(labels, return_counts=True)}")

In [ ]:
# CELLULE 5 - DATASET HYBRIDE AVEC PLUS DE VALIDATION
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset

print("🧠 Création du dataset hybride avec validation étendue...")

# Chargement tokenizer BioBERT
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")

class HybridMedicalDataset(Dataset):
    def __init__(self, texts, spark_features, labels, tokenizer, max_length=256):
        self.texts = texts
        self.spark_features = torch.FloatTensor(spark_features)
        self.labels = torch.LongTensor(labels)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text, truncation=True, padding='max_length',
            max_length=self.max_length, return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'spark_features': self.spark_features[idx],
            'labels': self.labels[idx]
        }

# Split des données AVEC PLUS DE VALIDATION
train_texts, val_texts, train_spark, val_spark, train_labels, val_labels = train_test_split(
    texts, spark_features, labels,
    test_size=0.3,  # ⬅️ 30% au lieu de 20% pour validation
    random_state=42,
    stratify=labels
)

# Création datasets
train_dataset = HybridMedicalDataset(train_texts, train_spark, train_labels, tokenizer)
val_dataset = HybridMedicalDataset(val_texts, val_spark, val_labels, tokenizer)

print(f"✅ Dataset créé - Train: {len(train_dataset)}, Val: {len(val_dataset)}")
print(f"📊 Ratio: {len(train_dataset)/len(texts)*100:.1f}% train, {len(val_dataset)/len(texts)*100:.1f}% val")

In [ ]:
# CELLULE 6f - SOLUTION ULTIME AVEC NOUVEAU NOM
import torch.nn as nn
from transformers import AutoModel

print("🏗️ Création du modèle avec nouveau nom...")

# NOUVEAU NOM COMPLÈTEMENT DIFFÉRENT
class MedicalClassifierV2(nn.Module):
    def __init__(self, n_classes=2, feature_size=400, drop_rate=0.3):
        super(MedicalClassifierV2, self).__init__()

        # BioBERT - CHARGEMENT DIRECT
        self.bert_model = AutoModel.from_pretrained("dmis-lab/biobert-v1.1")
        bert_dim = 768

        # Spark features - SIMPLIFIÉ
        self.feature_mapper = nn.Sequential(
            nn.Linear(feature_size, 128),
            nn.ReLU(),
            nn.Dropout(drop_rate)
        )

        # Classificateur
        total_dim = bert_dim + 128
        self.predictor = nn.Sequential(
            nn.Linear(total_dim, 256),
            nn.ReLU(),
            nn.Dropout(drop_rate),
            nn.Linear(256, n_classes)
        )

        print(f"✅ Modèle V2 créé: BERT({bert_dim}) + Features({feature_size})")

    def forward(self, input_ids, attention_mask, spark_features, labels=None):
        # BioBERT
        bert_output = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = bert_output.last_hidden_state[:, 0, :]

        # Features
        features_processed = self.feature_mapper(spark_features)

        # Fusion
        fused = torch.cat([cls_embedding, features_processed], dim=1)

        # Prédiction
        logits = self.predictor(fused)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {'loss': loss, 'logits': logits}

# INITIALISATION DIRECTE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Device: {device}")

# CRÉATION AVEC VALEURS DIRECTES
classifier_model = MedicalClassifierV2(n_classes=2, feature_size=400)
classifier_model.to(device)

print("🧪 Test du modèle...")
classifier_model.eval()
with torch.no_grad():
    sample_item = train_dataset[0]
    test_inputs = {
        'input_ids': sample_item['input_ids'].unsqueeze(0).to(device),
        'attention_mask': sample_item['attention_mask'].unsqueeze(0).to(device),
        'spark_features': sample_item['spark_features'].unsqueeze(0).to(device)
    }

    results = classifier_model(**test_inputs)
    print(f"✅ Test réussi! Shape logits: {results['logits'].shape}")
    probabilities = torch.softmax(results['logits'], dim=1).cpu().numpy()
    print(f"🎯 Probabilités: {probabilities}")
    print(f"🔮 Classe prédite: {np.argmax(probabilities)}")

print("🚀 Modèle V2 prêt pour l'entraînement!")

In [ ]:
# CELLULE 7 - CONFIGURATION ENTRAÎNEMENT AVANCÉE
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

print("⚡ Configuration de l'entraînement...")

def compute_metrics(eval_pred):
    """Calcule les métriques de performance"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Configuration d'entraînement optimisée
training_args = TrainingArguments(
    output_dir="./medical_classifier_results",
    overwrite_output_dir=True,

    # Hyperparamètres optimisés
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    warmup_steps=500,
    weight_decay=0.01,
    adam_epsilon=1e-8,

    # Stratégie d'évaluation
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",

    # Optimisations
    fp16=torch.cuda.is_available(),
    dataloader_pin_memory=False,
    gradient_accumulation_steps=1,

    # Logging
    logging_dir="./training_logs",
    logging_steps=50,
    report_to=None,

    # Sauvegarde
    save_total_limit=2,
    seed=42
)

print("✅ Configuration entraînement prête!")
print(f"📊 Epochs: {training_args.num_train_epochs}")
print(f"📦 Batch size: {training_args.per_device_train_batch_size}")
print(f"🎯 Learning rate: {training_args.learning_rate}")

In [ ]:
# CELLULE 8b - TRAINER CORRIGÉ
print("🎯 Création du trainer personnalisé CORRIGÉ...")

class HybridTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """Gère les features supplémentaires (version corrigée)"""
        labels = inputs.pop("labels")
        spark_features = inputs.pop("spark_features")

        outputs = model(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            spark_features=spark_features,
            labels=labels
        )

        return (outputs['loss'], outputs) if return_outputs else outputs['loss']

# Recréation du trainer avec la version corrigée
trainer = HybridTrainer(
    model=classifier_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

print("✅ Trainer CORRIGÉ créé avec succès!")

In [ ]:
# CELLULE 9 - LANCEMENT ENTRAÎNEMENT (REEXÉCUTER)
print("🚀 RELANCEMENT DE L'ENTRAÎNEMENT...")

# Démarrer l'entraînement
train_results = trainer.train()

# Sauvegarde du modèle final
trainer.save_model()
trainer.save_state()

print("✅ Entraînement terminé avec succès!")

# Affichage des résultats
print("\n📈 RÉSULTATS DE L'ENTRAÎNEMENT:")
print("=" * 50)
for key, value in train_results.metrics.items():
    print(f"{key:20}: {value:.4f}")

In [ ]:
# CELLULE TEST FINALE - 100% CORRIGÉE
import torch
import numpy as np

def predict(text):
    classifier_model.eval()   # ✅ CORRECTION ICI

    # --- 1) Pipeline Spark identique ---
    spark_df = spark.createDataFrame([(text,)], ["text"])
    spark_features_df = pipeline_model.transform(spark_df)
    pandas_features = spark_features_df.toPandas()

    # Extraire TF-IDF et Word2Vec
    tfidf = np.array([x.toArray() for x in pandas_features["tfidf_features"]])
    w2v   = np.array([x.toArray() for x in pandas_features["w2v_features"]])
    spark_features = np.concatenate([tfidf, w2v], axis=1)

    spark_tensor = torch.FloatTensor(spark_features).to(device)

    # --- 2) Encodage BERT identique ---
    encoding = tokenizer(
        text,
        return_tensors="pt",
        max_length=256,
        padding='max_length',
        truncation=True
    ).to(device)

    # --- 3) Prédiction correcte ---
    with torch.no_grad():
        outputs = classifier_model(
            input_ids=encoding["input_ids"],
            attention_mask=encoding["attention_mask"],
            spark_features=spark_tensor
        )

        logits = outputs["logits"]
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        pred = int(np.argmax(probs))

    return {
        "prediction": pred,
        "probabilities": probs
    }

print("✅ Fonction de prédiction opérationnelle.")


In [ ]:
predict("Medical investigation reveals relevant information regarding preventive measures.")


In [ ]:
# TESTS COMPLÉMENTAIRES - VERSION CORRIGÉE
test_cases = [
    "Clinical study shows positive outcomes for new treatment protocol in cardiovascular disease management",
    "Patient care methods evaluation demonstrates improved recovery rates with multidisciplinary approach",
    "Research data analysis reveals significant correlation between lifestyle factors and disease prevention",
    "Therapeutic approaches for chronic conditions show measurable benefits in long-term follow-up",
    "Medical procedures assessment indicates high efficacy rates with minimal side effects observed",
    "Healthcare strategies implementation results in better patient satisfaction and cost reduction",
    "Diagnostic methods innovation allows for earlier detection and more accurate classification",
    "Treatment evaluation protocol establishes clear guidelines for clinical decision making"
]

print("🧪 TESTS COMPLÉMENTAIRES DU MODÈLE")
print("=" * 60)

for i, test_text in enumerate(test_cases, 1):
    result = predict(test_text)
    prob_class_0 = result['probabilities'][0]
    prob_class_1 = result['probabilities'][1]
    prediction_label = "CLASSE 0" if result['prediction'] == 0 else "CLASSE 1"

    # CORRECTION : Utiliser np.max() au lieu de max()
    confidence = np.max([prob_class_0, prob_class_1])

    print(f"\nTest {i}:")
    print(f"Texte: {test_text[:80]}...")
    print(f"🔮 Prédiction: {prediction_label}")
    print(f"📊 Probabilités: [Classe 0: {prob_class_0:.3f}, Classe 1: {prob_class_1:.3f}]")
    print(f"📈 Confiance: {confidence:.1%}")
    print("-" * 50)

In [ ]:
# 🧪 BATTERIE ÉTENDUE DE TESTS
print("🧪 BATTERIE ÉTENDUE DE TESTS DU MODÈLE")
print("=" * 80)

# Catégorie 1: TEXTES MÉDICAUX POSITIFS
medical_positive = [
    "Le patient présente une amélioration significative après le traitement antibiotique",
    "Les résultats de l'essai clinique montrent une efficacité remarquable du nouveau médicament",
    "La chirurgie s'est déroulée sans complication avec une récupération rapide",
    "L'analyse sanguine confirme la guérison complète de l'infection",
    "La thérapie a réduit les symptômes de 80% chez la majorité des patients",
    "L'intervention précoce a permis d'éviter les complications sévères",
    "Le protocole de traitement démontre une supériorité statistique significative",
    "La qualité de vie des patients s'est améliorée de manière notable"
]

print("\n🏥 CATÉGORIE 1: TEXTES MÉDICAUX POSITIFS (FRANÇAIS)")
print("-" * 60)
for i, text in enumerate(medical_positive, 1):
    result = predict(text)
    confidence = np.max(result['probabilities'])
    prediction = "CLASSE 0" if result['prediction'] == 0 else "CLASSE 1"

    print(f"\nTest {i}: {text[:70]}...")
    print(f"   🎯 Prédiction: {prediction}")
    print(f"   📊 Confiance: {confidence:.1%}")
    print(f"   📈 Probas: [0: {result['probabilities'][0]:.3f}, 1: {result['probabilities'][1]:.3f}]")

In [ ]:
# Catégorie 2: TEXTES MÉDICAUX NÉGATIFS/NEUTRES
medical_negative = [
    "L'étude ne montre pas de différence significative entre les groupes traitement et placebo",
    "L'essai clinique a été arrêté prématurément en raison d'un manque d'efficacité",
    "Le patient ne présente aucune amélioration après 4 semaines de traitement",
    "Des effets secondaires graves ont été observés chez 20% des participants",
    "L'analyse statistique révèle un p-value non significatif de 0.45",
    "Le traitement n'a pas atteint le critère d'évaluation principal",
    "Aucun bénéfice clinique n'a été démontré dans cette population",
    "La mortalité est restée similaire entre les deux groupes d'intervention"
]

print("\n⚠️ CATÉGORIE 2: TEXTES MÉDICAUX NÉGATIFS/NEUTRES")
print("-" * 60)
for i, text in enumerate(medical_negative, 1):
    result = predict(text)
    confidence = np.max(result['probabilities'])
    prediction = "CLASSE 0" if result['prediction'] == 0 else "CLASSE 1"

    print(f"\nTest {i}: {text[:70]}...")
    print(f"   🎯 Prédiction: {prediction}")
    print(f"   📊 Confiance: {confidence:.1%}")
    print(f"   📈 Probas: [0: {result['probabilities'][0]:.3f}, 1: {result['probabilities'][1]:.3f}]")

In [ ]:
# Catégorie 3: TEXTES MÉDICAUX TECHNIQUES/SPÉCIALISÉS
medical_technical = [
    "L'IRM cérébrale met en évidence une lésion hyperintense en séquence T2",
    "La biopsie confirme un adénocarcinome bien différencié grade 2",
    "L'échocardiographie Doppler montre une fraction d'éjection à 55%",
    "Le scanner thoracique révèle des opacités en verre dépoli bilatérales",
    "L'analyse histologique objective une prolifération cellulaire modérée",
    "L'électroencéphalogramme détecte des ondes thêta anormales",
    "La ponction lombaire montre une hyperprotéinorachie à 0.8 g/L",
    "L'angiographie coronarienne objective une sténose à 70% de l'artère interventriculaire antérieure"
]

print("\n🔬 CATÉGORIE 3: TEXTES MÉDICAUX TECHNIQUES/SPÉCIALISÉS")
print("-" * 60)
for i, text in enumerate(medical_technical, 1):
    result = predict(text)
    confidence = np.max(result['probabilities'])
    prediction = "CLASSE 0" if result['prediction'] == 0 else "CLASSE 1"

    print(f"\nTest {i}: {text[:70]}...")
    print(f"   🎯 Prédiction: {prediction}")
    print(f"   📊 Confiance: {confidence:.1%}")
    print(f"   📈 Probas: [0: {result['probabilities'][0]:.3f}, 1: {result['probabilities'][1]:.3f}]")

In [ ]:
# TESTS AVANCÉS - TEXTES MÉDICAUX COMPLEXES
complex_medical_tests = [
    "Randomized controlled trial demonstrates that the combination therapy of metformin and lifestyle intervention significantly reduces HbA1c levels in prediabetic patients over 24-month follow-up period with p-value < 0.001",
    "Systematic review and meta-analysis of 15 studies involving 50,000 participants shows no significant association between moderate coffee consumption and increased cardiovascular risk, with heterogeneity I² = 25%",
    "Retrospective cohort study using electronic health records from 2010-2020 indicates that early ambulation post-surgery reduces hospital length of stay by 1.8 days (95% CI: 1.2-2.4) without increasing complication rates",
    "Multicenter prospective observational study finds that telemedicine implementation during pandemic maintained quality of care for chronic disease management while improving patient accessibility (OR: 2.3, p=0.01)"
]

print("\n🔬 TESTS AVEC TEXTES MÉDICAUX COMPLEXES")
print("=" * 60)

for i, text in enumerate(complex_medical_tests, 1):
    result = predict(text)
    prob_class_0 = result['probabilities'][0]
    prob_class_1 = result['probabilities'][1]
    confidence = np.max([prob_class_0, prob_class_1])

    print(f"\nTest Complexe {i}:")
    print(f"Texte: {text[:70]}...")
    print(f"🔮 Prédiction: {'CLASSE 0' if result['prediction'] == 0 else 'CLASSE 1'}")
    print(f"📊 Probabilités: [Classe 0: {prob_class_0:.3f}, Classe 1: {prob_class_1:.3f}]")
    print(f"📈 Confiance: {confidence:.1%}")

In [ ]:
# TESTS DE ROBUSTESSE FINAUX
edge_cases = [
    "",  # texte vide
    " ",  # espace seul
    "A B C D E",  # lettres sans sens
    "Patient patient patient patient",  # répétition
    "1234567890",  # chiffres seulement
    "COVID-19 SARS-CoV-2 ICU ventilator",  # termes médicaux courts
]

print("\n🧪 TESTS DE ROBUSTESSE (CAS EXTRÊMES)")
print("=" * 50)

for i, text in enumerate(edge_cases, 1):
    try:
        result = predict(text)
        prob_class_0 = result['probabilities'][0]
        prob_class_1 = result['probabilities'][1]
        confidence = np.max([prob_class_0, prob_class_1])

        print(f"\nCas {i}: '{text if text else 'VIDE'}'")
        print(f"🔮 Prédiction: {'CLASSE 0' if result['prediction'] == 0 else 'CLASSE 1'}")
        print(f"📊 Probabilités: [Classe 0: {prob_class_0:.3f}, Classe 1: {prob_class_1:.3f}]")
        print(f"📈 Confiance: {confidence:.1%}")

    except Exception as e:
        print(f"\nCas {i} '{text}': ERREUR - {str(e)[:100]}")

In [ ]:
# 🧪 BATTERIE COMPLÈTE DE TESTS
print("🧪 BATTERIE COMPLÈTE DE TESTS DU MODÈLE")
print("=" * 70)

# Catégorie 1: TEXTES MÉDICAUX STANDARDS
medical_standard = [
    "Patient shows significant improvement after treatment with antibiotics",
    "Clinical trial results indicate positive outcomes for new drug therapy",
    "Medical examination reveals normal vital signs and stable condition",
    "Surgical procedure completed successfully with no complications",
    "Laboratory tests confirm diagnosis of bacterial infection",
    "Treatment protocol effective in reducing symptoms by 70%",
    "Healthcare intervention improves patient quality of life",
    "Preventive measures reduce disease incidence by 45%"
]

print("\n🔬 CATÉGORIE 1: TEXTES MÉDICAUX STANDARDS")
print("-" * 50)
for i, text in enumerate(medical_standard, 1):
    result = predict(text)
    confidence = np.max(result['probabilities'])
    prediction = "CLASSE 0" if result['prediction'] == 0 else "CLASSE 1"

    print(f"\nTest {i}: {text[:60]}...")
    print(f"   🎯 Prédiction: {prediction}")
    print(f"   📊 Confiance: {confidence:.1%}")
    print(f"   📈 Probas: [0: {result['probabilities'][0]:.3f}, 1: {result['probabilities'][1]:.3f}]")

In [ ]:
# Catégorie 2: TEXTES MÉDICAUX COMPLEXES
medical_complex = [
    "Randomized double-blind placebo-controlled trial demonstrates statistically significant improvement in primary endpoint (p<0.001) with hazard ratio of 0.65 for cardiovascular events",
    "Systematic review and meta-analysis of 25 RCTs involving 150,000 participants shows relative risk reduction of 22% (95% CI: 15-29%) for mortality outcomes",
    "Multicenter prospective cohort study with 5-year follow-up indicates dose-response relationship between intervention intensity and clinical benefits",
    "Real-world evidence from electronic health records analysis suggests 30% lower readmission rates with integrated care model",
    "Cost-effectiveness analysis reveals incremental cost-effectiveness ratio of $45,000 per QALY gained for novel therapeutic approach",
    "Subgroup analysis shows enhanced treatment effects in patients with specific genetic markers (OR: 2.3, p=0.01)"
]

print("\n🔍 CATÉGORIE 2: TEXTES MÉDICAUX COMPLEXES (STATISTIQUES)")
print("-" * 50)
for i, text in enumerate(medical_complex, 1):
    result = predict(text)
    confidence = np.max(result['probabilities'])
    prediction = "CLASSE 0" if result['prediction'] == 0 else "CLASSE 1"

    print(f"\nTest {i}: {text[:70]}...")
    print(f"   🎯 Prédiction: {prediction}")
    print(f"   📊 Confiance: {confidence:.1%}")
    print(f"   📈 Probas: [0: {result['probabilities'][0]:.3f}, 1: {result['probabilities'][1]:.3f}]")

In [ ]:
# TEST 1: ANALYSE DES PERFORMANCES TRAIN vs VALIDATION (CORRIGÉ)
print("📊 TEST 1: COMPARAISON TRAIN/VALIDATION")
print("=" * 60)

# Évaluation sur l'ensemble d'entraînement
print("🔍 Évaluation sur l'ensemble d'entraînement...")
train_predictions = []
train_actuals = []
train_confidences = []

# CORRECTION : Éviter la fonction min() problématique
train_sample_size = 200 if len(train_dataset) > 200 else len(train_dataset)
for i in range(train_sample_size):
    sample = train_dataset[i]
    text = train_texts[i]

    result = predict(text)
    train_predictions.append(result['prediction'])
    train_actuals.append(sample['labels'].item())
    train_confidences.append(np.max(result['probabilities']))

train_accuracy = np.mean(np.array(train_predictions) == np.array(train_actuals))
train_avg_confidence = np.mean(train_confidences)

print(f"✅ Performance entraînement:")
print(f"   • Accuracy: {train_accuracy:.1%}")
print(f"   • Confiance moyenne: {train_avg_confidence:.1%}")
print(f"   • Échantillon: {train_sample_size} textes")

# Évaluation sur l'ensemble de validation
print("\n🔍 Évaluation sur l'ensemble de validation...")
val_predictions = []
val_actuals = []
val_confidences = []

# CORRECTION : Même approche pour validation
val_sample_size = 200 if len(val_dataset) > 200 else len(val_dataset)
for i in range(val_sample_size):
    sample = val_dataset[i]
    text = val_texts[i]

    result = predict(text)
    val_predictions.append(result['prediction'])
    val_actuals.append(sample['labels'].item())
    val_confidences.append(np.max(result['probabilities']))

val_accuracy = np.mean(np.array(val_predictions) == np.array(val_actuals))
val_avg_confidence = np.mean(val_confidences)

print(f"✅ Performance validation:")
print(f"   • Accuracy: {val_accuracy:.1%}")
print(f"   • Confiance moyenne: {val_avg_confidence:.1%}")
print(f"   • Échantillon: {val_sample_size} textes")

# Calcul du gap d'overfitting
accuracy_gap = train_accuracy - val_accuracy
confidence_gap = train_avg_confidence - val_avg_confidence

print(f"\n🎯 ANALYSE OVERFITTING:")
print(f"   • Écart d'accuracy: {accuracy_gap:+.3f} ({accuracy_gap*100:+.1f}%)")
print(f"   • Écart de confiance: {confidence_gap:+.3f} ({confidence_gap*100:+.1f}%)")

# Interprétation
if accuracy_gap < 0.02:
    overfitting_status = "✅ TRÈS FAIBLE - Excellente généralisation"
elif accuracy_gap < 0.05:
    overfitting_status = "✅ FAIBLE - Bonne généralisation"
elif accuracy_gap < 0.10:
    overfitting_status = "⚠️  MODÉRÉ - Surveillance recommandée"
else:
    overfitting_status = "❌ ÉLEVÉ - Problème de généralisation"

print(f"   • Statut overfitting: {overfitting_status}")

In [ ]:
# TEST DE GÉNÉRALISATION - VERSION COMPLÈTEMENT ISOLÉE
print("🔍 TESTS DE GÉNÉRALISATION - VERSION SÉCURISÉE")
print("=" * 70)

# TEST 1: PERFORMANCE SIMPLIFIÉE
print("\n📊 TEST 1: PERFORMANCE DE BASE")
print("-" * 50)

def safe_evaluate_dataset(dataset, texts, sample_size):
    """Évaluation sécurisée sans conflit PySpark"""
    predictions = []
    actuals = []
    confidences = []

    for i in range(sample_size):
        sample = dataset[i]
        text = texts[i]

        result = predict(text)
        # Conversion manuelle en types natifs
        pred_val = int(result['prediction'])
        actual_val = int(sample['labels'].item())
        conf_val = float(result['probabilities'][0])  # Utiliser directement le tableau

        predictions.append(pred_val)
        actuals.append(actual_val)
        confidences.append(conf_val)

    # Calcul manuel de l'accuracy
    correct = 0
    for i in range(len(predictions)):
        if predictions[i] == actuals[i]:
            correct += 1
    accuracy = correct / len(predictions)

    # Calcul manuel de la confiance moyenne
    total_conf = 0
    for conf in confidences:
        total_conf += conf
    avg_confidence = total_conf / len(confidences)

    return accuracy, avg_confidence

# Évaluation avec calculs manuels
print("🔍 Évaluation sur échantillon limité...")
train_acc, train_conf = safe_evaluate_dataset(train_dataset, train_texts, 100)
val_acc, val_conf = safe_evaluate_dataset(val_dataset, val_texts, 100)

print(f"📊 RÉSULTATS:")
print(f"   • Train - Accuracy: {train_acc:.1%}, Confiance: {train_conf:.1%}")
print(f"   • Validation - Accuracy: {val_acc:.1%}, Confiance: {val_conf:.1%}")

# Analyse overfitting manuelle
acc_gap = train_acc - val_acc
conf_gap = train_conf - val_conf

print(f"\n🎯 ANALYSE OVERFITTING:")
print(f"   • Écart accuracy: {acc_gap:.3f}")
print(f"   • Écart confiance: {conf_gap:.3f}")

if acc_gap < 0.03:
    status = "✅ FAIBLE OVERFITTING"
elif acc_gap < 0.07:
    status = "⚠️  OVERFITTING MODÉRÉ"
else:
    status = "❌ FORT OVERFITTING"

print(f"   • Statut: {status}")

In [ ]:
# TEST 2: COMPORTEMENT SUR NOUVEAUX TEXTES
print("\n🎲 TEST 2: TEXTES NOUVEAUX")
print("-" * 50)

test_texts = [
    "Medical treatment effective for patients",
    "Clinical trial shows positive results",
    "Therapy improves health outcomes",
    "Healthcare intervention successful",
    "Patient recovery progressing well"
]

print("🧪 Test avec 5 textes nouveaux...")

class_0_count = 0
class_1_count = 0
total_conf = 0

for i, text in enumerate(test_texts):
    result = predict(text)
    pred = int(result['prediction'])
    prob_0 = float(result['probabilities'][0])
    prob_1 = float(result['probabilities'][1])

    # Calcul manuel confidence
    if prob_0 > prob_1:
        conf = prob_0
    else:
        conf = prob_1

    if pred == 0:
        class_0_count += 1
    else:
        class_1_count += 1

    total_conf += conf

    print(f"   • Texte {i+1}: Classe {pred} ({conf:.1%})")

avg_conf = total_conf / len(test_texts)
print(f"\n📊 RÉSUMÉ:")
print(f"   • Classe 0: {class_0_count}/5")
print(f"   • Classe 1: {class_1_count}/5")
print(f"   • Confiance moyenne: {avg_conf:.1%}")

if class_0_count >= 4:
    coherency = "✅ TRÈS COHÉRENT"
elif class_0_count >= 3:
    coherency = "✅ COHÉRENT"
else:
    coherency = "⚠️  PEU COHÉRENT"

print(f"   • Cohérence: {coherency}")

In [ ]:
# TEST 3: STABILITÉ DES PRÉDICTIONS
print("\n🔄 TEST 3: TEST DE STABILITÉ")
print("-" * 50)

test_text = "Medical study demonstrates treatment efficacy"
print(f"🧪 Test de stabilité: '{test_text}'")

predictions = []
confidences = []

for i in range(5):
    result = predict(test_text)
    pred = int(result['prediction'])
    prob_0 = float(result['probabilities'][0])
    prob_1 = float(result['probabilities'][1])

    if prob_0 > prob_1:
        conf = prob_0
    else:
        conf = prob_1

    predictions.append(pred)
    confidences.append(conf)
    print(f"   • Essai {i+1}: Classe {pred} ({conf:.1%})")

# Analyse manuelle de la stabilité
first_pred = predictions[0]
all_same = True
for p in predictions:
    if p != first_pred:
        all_same = False
        break

# Calcul manuel variation confiance
min_conf = confidences[0]
max_conf = confidences[0]
for c in confidences:
    if c < min_conf:
        min_conf = c
    if c > max_conf:
        max_conf = c
conf_range = max_conf - min_conf

print(f"\n📊 STABILITÉ:")
print(f"   • Prédictions identiques: {'✅ OUI' if all_same else '❌ NON'}")
print(f"   • Variation confiance: {conf_range:.3f}")

if all_same and conf_range < 0.02:
    stability = "✅ EXCELLENTE"
elif all_same and conf_range < 0.05:
    stability = "✅ BONNE"
else:
    stability = "⚠️  VARIABLE"

print(f"   • Stabilité: {stability}")

In [ ]:
# TEST 4: COMPORTEMENT TEXTES DIFFICILES
print("\n🎯 TEST 4: TEXTES AMBIGUS")
print("-" * 50)

ambiguous_texts = [
    "Results are inconclusive and need more study",
    "Statistical analysis shows borderline significance",
    "Mixed outcomes observed in different groups",
    "Further research required for confirmation",
    "Benefits remain uncertain despite some improvement"
]

print("🧪 Test avec 5 textes ambigus...")

low_conf_count = 0
total_conf_amb = 0

for i, text in enumerate(ambiguous_texts):
    result = predict(text)
    pred = int(result['prediction'])
    prob_0 = float(result['probabilities'][0])
    prob_1 = float(result['probabilities'][1])

    if prob_0 > prob_1:
        conf = prob_0
    else:
        conf = prob_1

    total_conf_amb += conf

    if conf < 0.6:
        low_conf_count += 1
        caution_level = "PRUDENT"
    elif conf < 0.7:
        caution_level = "MODÉRÉ"
    else:
        caution_level = "CONFIANT"

    print(f"   • Texte {i+1}: Classe {pred} ({conf:.1%}) - {caution_level}")

avg_conf_amb = total_conf_amb / len(ambiguous_texts)
print(f"\n📊 COMPORTEMENT AMBIGU:")
print(f"   • Confiance moyenne: {avg_conf_amb:.1%}")
print(f"   • Textes peu confiants: {low_conf_count}/5")

if avg_conf_amb < 0.65:
    prudence = "✅ PRUDENT"
elif avg_conf_amb < 0.75:
    prudence = "⚠️  MODÉRÉMENT CONFIANT"
else:
    prudence = "❌ TROP CONFIANT"

print(f"   • Prudence: {prudence}")

In [ ]:
print("🔧 CORRECTION DU NOM DU SECRET...")

from google.colab import userdata
import os

# Maintenant on utilise le BON nom de secret
try:
    # Votre secret s'appelle "gsk_votre_cle_api_ici"
    groq_key = userdata.get('gsk_votre_cle_api_ici')

    if groq_key and groq_key.startswith('gsk_'):
        os.environ['GROQ_API_KEY'] = groq_key
        print("✅ Clé Groq chargée depuis les secrets Colab!")


    else:
        print("❌ La valeur du secret 'gsk_votre_cle_api_ici' n'est pas une vraie clé Groq")
        print("💡 Vous devez mettre votre VRAIE clé Groq dans ce secret")

except Exception as e:
    print(f"❌ Erreur: {e}")

In [ ]:
print("⚡ SYSTÈME GROQ AVEC VOTRE SECRET ACTUEL...")

import requests
import json
import os
from typing import Dict, Any

class GroqCounterArgument:
    def __init__(self):
        # Utiliser le bon nom de variable d'environnement
        self.api_key = os.getenv('GROQ_API_KEY')

        if not self.api_key:
            raise ValueError("❌ GROQ_API_KEY non trouvée - vérifiez la configuration")

        self.base_url = "https://api.groq.com/openai/v1/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        self.expert_prompt = """
Vous êtes un expert médical. Réfutez les fake news COVID avec des arguments scientifiques solides.

EXEMPLES:
Affirmation: "Les vaccins modifient l'ADN"
Réponse: "Faux. Les vaccins à ARNm ne pénètrent pas dans le noyau cellulaire. Source: NIH."

Affirmation: "La 5G propage le COVID"
Réponse: "Impossible scientifiquement. Transmission par gouttelettes. Source: OMS."
"""

    def generate_counter_argument(self, fake_news_text: str, confidence: float) -> Dict[str, Any]:
        print(f"⚡ Groq: '{fake_news_text}'")

        try:
            payload = {
                "messages": [
                    {"role": "system", "content": self.expert_prompt},
                    {"role": "user", "content": f"Réfutez scientifiquement: '{fake_news_text}' - soyez factuel et citez des sources"}
                ],
                "model": "llama3-8b-8192",
                "temperature": 0.7,
                "max_tokens": 300,
                "top_p": 0.9
            }

            response = requests.post(self.base_url, headers=self.headers, json=payload, timeout=15)

            if response.status_code == 200:
                result = response.json()
                counter_text = result['choices'][0]['message']['content'].strip()

                return {
                    'original_claim': fake_news_text,
                    'detection_confidence': f"{confidence:.1%}",
                    'verdict': "🚨 FAKE NEWS DÉTECTÉE",
                    'counter_argument': counter_text,
                    'response_time': f"{response.elapsed.total_seconds():.2f}s",
                    'model_used': "Groq-Llama3",
                    'sources': ["Groq AI + Sources scientifiques"],
                    'status': "✅ RÉPONSE INTELLIGENTE"
                }
            else:
                print(f"❌ Erreur API: {response.status_code}")
                return self._fallback_response(fake_news_text, confidence)

        except Exception as e:
            print(f"❌ Exception: {e}")
            return self._fallback_response(fake_news_text, confidence)

    def _fallback_response(self, claim: str, confidence: float):
        return {
            'original_claim': claim,
            'detection_confidence': f"{confidence:.1%}",
            'verdict': "⚠️ INFORMATION ERRONÉE",
            'counter_argument': f"Affirmation scientifiquement infondée: '{claim}'. Consultez l'OMS pour des informations vérifiées.",
            'response_time': "0s",
            'model_used': "Système de base",
            'sources': ["OMS"],
            'status': "❌ GROQ INDISPONIBLE"
        }

# Initialisation avec VOTRE configuration actuelle
print("🔧 Initialisation avec votre secret...")

try:
    groq_key = os.getenv('GROQ_API_KEY')
    if groq_key:
        counter_system = GroqCounterArgument()
        print("✅ Système Groq activé avec votre secret!")
        print(f"🔐 Utilisation du secret: gsk_votre_cle_api_ici")
    else:
        raise ValueError("Clé non configurée")

except Exception as e:
    print(f"⚠️ Impossible d'activer Groq: {e}")
    # On utilise le système amélioré en fallback
    from advanced_medical_counter import AdvancedMedicalCounter
    counter_system = AdvancedMedicalCounter()
    print("✅ Système médical avancé activé (fallback)")

print("🎯 Prêt pour la contre-argumentation!")

In [ ]:
print("🔧 CORRECTION DES ERREURS...")

import requests
import json
import os
from typing import Dict, Any

class GroqCounterArgument:
    def __init__(self):
        self.api_key = os.getenv('GROQ_API_KEY')

        if not self.api_key:
            raise ValueError("❌ GROQ_API_KEY non trouvée")

        self.base_url = "https://api.groq.com/openai/v1/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        self.expert_prompt = """
Vous êtes un expert médical. Réfutez les fake news COVID avec des arguments scientifiques.
"""

    def generate_counter_argument(self, fake_news_text: str, confidence: float) -> Dict[str, Any]:
        print(f"⚡ Groq: '{fake_news_text}'")

        try:
            # Payload corrigé pour éviter l'erreur 400
            payload = {
                "messages": [
                    {
                        "role": "system",
                        "content": self.expert_prompt
                    },
                    {
                        "role": "user",
                        "content": f"Réfutez scientifiquement cette affirmation: {fake_news_text}"
                    }
                ],
                "model": "llama3-8b-8192",
                "max_tokens": 350,
                "temperature": 0.7
            }

            response = requests.post(self.base_url, headers=self.headers, json=payload, timeout=15)

            if response.status_code == 200:
                result = response.json()
                counter_text = result['choices'][0]['message']['content'].strip()

                return {
                    'original_claim': fake_news_text,
                    'detection_confidence': f"{confidence:.1%}",
                    'verdict': "🚨 FAKE NEWS DÉTECTÉE",
                    'counter_argument': counter_text,
                    'response_time': f"{response.elapsed.total_seconds():.2f}s",
                    'model_used': "Groq-Llama3",
                    'sources': ["Groq AI + Sources scientifiques"],
                    'recommendation': "Consultez l'OMS (who.int) pour des informations vérifiées.",
                    'status': "✅ RÉPONSE INTELLIGENTE"
                }
            else:
                print(f"❌ Erreur API {response.status_code}: {response.text[:200]}")
                return self._fallback_response(fake_news_text, confidence)

        except Exception as e:
            print(f"❌ Exception: {e}")
            return self._fallback_response(fake_news_text, confidence)

    def _fallback_response(self, claim: str, confidence: float):
        """Réponse de secours avec TOUTES les clés nécessaires"""
        return {
            'original_claim': claim,
            'detection_confidence': f"{confidence:.1%}",
            'verdict': "⚠️ INFORMATION ERRONÉE",
            'counter_argument': f"Affirmation scientifiquement infondée: '{claim}'. Consultez l'OMS pour des informations vérifiées.",
            'response_time': "0s",
            'model_used': "Système de base",
            'sources': ["OMS"],
            'recommendation': "Sources fiables: OMS (who.int) | Santé France (sante.fr)",
            'status': "❌ GROQ INDISPONIBLE"
        }

# Système médical avancé pour fallback
class AdvancedMedicalCounter:
    def __init__(self):
        self.knowledge_base = {
            'magnétique': {
                'response': "❌ FAUX. Les vaccins COVID ne rendent pas magnétique. Composition: ARNm, lipides, sels, sucre. Aucun métal magnétique. Sources: OMS, FDA.",
                'sources': ['OMS', 'FDA']
            },
            'bill gates': {
                'response': "❌ FAUX. Bill Gates n'a pas breveté le coronavirus. Les virus naturels ne sont pas brevetables. Source: OMS.",
                'sources': ['OMS']
            },
            '5g': {
                'response': "❌ FAUX. La 5G ne crée pas de virus. Transmission par gouttelettes uniquement. Aucun lien scientifique. Source: OMS.",
                'sources': ['OMS']
            },
            'masque': {
                'response': "❌ FAUX. Les masques réduisent la transmission sans causer d'infections fongiques. Efficacité prouvée. Source: The Lancet.",
                'sources': ['The Lancet']
            }
        }

    def generate_counter_argument(self, fake_news_text, confidence):
        text_lower = fake_news_text.lower()

        for keyword, data in self.knowledge_base.items():
            if keyword in text_lower:
                return {
                    'original_claim': fake_news_text,
                    'detection_confidence': f"{confidence:.1%}",
                    'verdict': "🚨 FAKE NEWS DÉTECTÉE",
                    'counter_argument': data['response'],
                    'response_time': "0.01s",
                    'model_used': "Base de connaissances médicale",
                    'sources': data['sources'],
                    'recommendation': "💡 Sources: OMS (who.int) | CDC (cdc.gov)",
                    'status': "✅ RÉPONSE SPÉCIFIQUE"
                }

        # Réponse par défaut
        return {
            'original_claim': fake_news_text,
            'detection_confidence': f"{confidence:.1%}",
            'verdict': "⚠️ INFORMATION ERRONÉE",
            'counter_argument': f"Cette affirmation est scientifiquement infondée. Consultez l'OMS pour des informations vérifiées.",
            'response_time': "0.01s",
            'model_used': "Système de base",
            'sources': ["OMS"],
            'recommendation': "Sources fiables: OMS (who.int)",
            'status': "⚠️ RÉPONSE GÉNÉRIQUE"
        }

# Initialisation
print("🔧 Initialisation du système...")

try:
    groq_key = os.getenv('GROQ_API_KEY')
    if groq_key and groq_key.startswith('gsk_'):
        counter_system = GroqCounterArgument()
        print("✅ Système Groq activé!")
    else:
        raise ValueError("Clé Groq invalide")

except Exception as e:
    print(f"⚠️ Groq impossible: {e}")
    counter_system = AdvancedMedicalCounter()
    print("✅ Système médical avancé activé")

print("🎯 Prêt pour la contre-argumentation!")

In [ ]:
print("🔄 MISE À JOUR DES MODÈLES GROQ...")

import requests
import json
import os
from typing import Dict, Any

class UpdatedGroqCounterArgument:
    def __init__(self):
        self.api_key = os.getenv('GROQ_API_KEY')

        if not self.api_key:
            raise ValueError("❌ GROQ_API_KEY non trouvée")

        self.base_url = "https://api.groq.com/openai/v1/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        # Nouveaux modèles Groq disponibles
        self.available_models = [
            "llama-3.1-8b-instant",    # Rapide et efficace
            "llama-3.1-70b-versatile", # Plus puissant
            "mixtral-8x7b-32768",      # Très performant
            "gemma2-9b-it"             # Alternatif Google
        ]

        self.current_model = self.available_models[0]  # Utiliser le premier disponible

        self.expert_prompt = """
Vous êtes un expert médical et scientifique. Réfutez les fake news COVID avec des arguments factuels, des sources vérifiées (OMS, CDC, études scientifiques) et un ton pédagogique.

Fournissez des contre-arguments scientifiques précis et cités.
"""

    def generate_counter_argument(self, fake_news_text: str, confidence: float) -> Dict[str, Any]:
        print(f"⚡ Groq [{self.current_model}]: '{fake_news_text}'")

        try:
            payload = {
                "messages": [
                    {
                        "role": "system",
                        "content": self.expert_prompt
                    },
                    {
                        "role": "user",
                        "content": f"Réfutez scientifiquement cette fake news médicale: \"{fake_news_text}\". Soyez factuel, citez des sources et expliquez simplement."
                    }
                ],
                "model": self.current_model,
                "max_tokens": 400,
                "temperature": 0.7,
                "top_p": 0.9
            }

            response = requests.post(self.base_url, headers=self.headers, json=payload, timeout=20)

            if response.status_code == 200:
                result = response.json()
                counter_text = result['choices'][0]['message']['content'].strip()

                return {
                    'original_claim': fake_news_text,
                    'detection_confidence': f"{confidence:.1%}",
                    'verdict': "🚨 FAKE NEWS DÉTECTÉE",
                    'counter_argument': counter_text,
                    'response_time': f"{response.elapsed.total_seconds():.2f}s",
                    'model_used': f"Groq-{self.current_model}",
                    'sources': ["IA Groq + Connaissances scientifiques"],
                    'recommendation': "💡 Pour vérifier: OMS (who.int) | CDC (cdc.gov) | Santé France (sante.fr)",
                    'status': "✅ RÉPONSE INTELLIGENTE GROQ"
                }
            else:
                print(f"❌ Erreur {response.status_code}: {response.text[:150]}")
                # Essayer un autre modèle si erreur
                return self._try_alternative_models(fake_news_text, confidence)

        except Exception as e:
            print(f"❌ Exception: {e}")
            return self._fallback_response(fake_news_text, confidence)

    def _try_alternative_models(self, fake_news_text: str, confidence: float):
        """Essaye d'autres modèles disponibles"""
        print("🔄 Essai des modèles alternatifs...")

        for model in self.available_models[1:]:  # Essayer les autres modèles
            print(f"  🔄 Test avec {model}...")
            try:
                payload = {
                    "messages": [
                        {"role": "system", "content": self.expert_prompt},
                        {"role": "user", "content": f"Réfutez: \"{fake_news_text}\""}
                    ],
                    "model": model,
                    "max_tokens": 300,
                    "temperature": 0.7
                }

                response = requests.post(self.base_url, headers=self.headers, json=payload, timeout=15)

                if response.status_code == 200:
                    result = response.json()
                    counter_text = result['choices'][0]['message']['content'].strip()
                    self.current_model = model  # Mettre à jour le modèle actuel

                    return {
                        'original_claim': fake_news_text,
                        'detection_confidence': f"{confidence:.1%}",
                        'verdict': "🚨 FAKE NEWS DÉTECTÉE",
                        'counter_argument': counter_text,
                        'response_time': f"{response.elapsed.total_seconds():.2f}s",
                        'model_used': f"Groq-{model}",
                        'sources': ["IA Groq + Sources médicales"],
                        'recommendation': "💡 Vérifiez sur: OMS (who.int)",
                        'status': f"✅ RÉPONSE AVEC {model}"
                    }

            except Exception as e:
                print(f"    ❌ {model} échoué: {e}")
                continue

        # Si tous les modèles échouent
        return self._fallback_response(fake_news_text, confidence)

    def _fallback_response(self, claim: str, confidence: float):
        return {
            'original_claim': claim,
            'detection_confidence': f"{confidence:.1%}",
            'verdict': "⚠️ INFORMATION ERRONÉE",
            'counter_argument': f"Cette affirmation est scientifiquement infondée. Consultez des sources officielles pour des informations vérifiées.",
            'response_time': "0s",
            'model_used': "Système de base",
            'sources': ["OMS"],
            'recommendation': "🔍 Sources fiables: OMS (who.int) | CDC (cdc.gov)",
            'status': "❌ GROQ INDISPONIBLE"
        }

# Système médical avancé amélioré
class EnhancedMedicalCounter:
    def __init__(self):
        self.knowledge_base = {
            'magnétique': {
                'response': "❌ FAUX. Les vaccins COVID ne rendent pas magnétique. Composition: ARNm, lipides, sels stabilisateurs, sucre. Aucun métal magnétique. Les ingrédients sont publics et vérifiés par l'EMA et la FDA. Sources: European Medicines Agency, US Food and Drug Administration.",
                'sources': ['EMA', 'FDA', 'OMS']
            },
            'bill gates': {
                'response': "❌ FAUX. Bill Gates n'a pas breveté le coronavirus SARS-CoV-2. Les virus naturels ne sont pas brevetables. Le génome du virus a été séquencé et partagé internationalement. Sources: OMS, Nature, GISAID.",
                'sources': ['OMS', 'Nature', 'GISAID']
            },
            '5g': {
                'response': "❌ FAUX. La 5G ne crée pas et ne propage pas les virus. Les coronavirus se transmettent par gouttelettes respiratoires et contacts. Aucun mécanisme biologique ne relie les ondes radio aux virus. Sources: Organisation Mondiale de la Santé, Commission internationale de protection contre les rayonnements non ionisants.",
                'sources': ['OMS', 'ICNIRP', 'IEEE']
            },
            'masque': {
                'response': "❌ FAUX. Les masques ne causent pas d'infections fongiques quand utilisés correctement et changés régulièrement. Ils réduisent la transmission des gouttelettes de 70-90%. Études: The Lancet, Journal of Hospital Infection, CDC.",
                'sources': ['The Lancet', 'CDC', 'Journal of Hospital Infection']
            },
            'hydroxychloroquine': {
                'response': "❌ FAUX. L'hydroxychloroquine n'a pas démontré d'efficacité contre le COVID-19 dans les essais cliniques randomisés (RECOVERY, SOLIDARITY). Les traitements recommandés sont basés sur des preuves scientifiques. Sources: OMS, New England Journal of Medicine, The Lancet.",
                'sources': ['OMS', 'NEJM', 'The Lancet']
            },
            'test pcr': {
                'response': "❌ FAUX. Les tests PCR sont extrêmement spécifiques (>99%) et sensibles. C'est la méthode de référence en virologie depuis 30 ans. La détection de l'ARN viral est fiable et standardisée. Sources: OMS, Journal of Clinical Microbiology, FDA.",
                'sources': ['OMS', 'Journal of Clinical Microbiology', 'FDA']
            }
        }

    def generate_counter_argument(self, fake_news_text, confidence):
        text_lower = fake_news_text.lower()

        for keyword, data in self.knowledge_base.items():
            if keyword in text_lower:
                return {
                    'original_claim': fake_news_text,
                    'detection_confidence': f"{confidence:.1%}",
                    'verdict': "🚨 FAKE NEWS DÉTECTÉE",
                    'counter_argument': data['response'],
                    'response_time': "0.01s",
                    'model_used': "Base de connaissances médicale avancée",
                    'sources': data['sources'],
                    'recommendation': "💡 Sources vérifiées: OMS (who.int) | CDC (cdc.gov) | Santé France (sante.fr)",
                    'status': "✅ RÉPONSE SCIENTIFIQUE PRÉCISE"
                }

        # Réponse intelligente pour les cas non couverts
        return {
            'original_claim': fake_news_text,
            'detection_confidence': f"{confidence:.1%}",
            'verdict': "⚠️ INFORMATION NON VÉRIFIÉE",
            'counter_argument': f"Cette affirmation ne correspond pas aux connaissances médicales établies. Les informations vérifiées sont disponibles auprès des autorités sanitaires officielles.",
            'response_time': "0.01s",
            'model_used': "Système expert médical",
            'sources': ["Organisation Mondiale de la Santé"],
            'recommendation': "🔍 Consultez: OMS (who.int) pour des informations médicales validées",
            'status': "⚠️ RÉPONSE GÉNÉRIQUE"
        }

# Initialisation avec les nouveaux modèles
print("🔧 Initialisation du système mis à jour...")

try:
    groq_key = os.getenv('GROQ_API_KEY')
    if groq_key and groq_key.startswith('gsk_'):
        counter_system = UpdatedGroqCounterArgument()
        print("✅ Système Groq mis à jour avec les nouveaux modèles!")
        print(f"🔧 Modèles disponibles: {', '.join(counter_system.available_models)}")
    else:
        raise ValueError("Clé Groq invalide")

except Exception as e:
    print(f"⚠️ Groq impossible: {e}")
    counter_system = EnhancedMedicalCounter()
    print("✅ Système médical avancé activé (réponses scientifiques précises)")

print("🎯 Système de contre-argumentation prêt!")

In [ ]:
print("🧪 TEST AVEC LES NOUVEAUX MODÈLES GROQ...")

test_cases = [
    ("Les vaccins COVID rendent magnétique", 0.94),
    ("Bill Gates a breveté le coronavirus", 0.91),
    ("La 5G crée le virus par résonance", 0.93),
]

print(f"🧪 Test de {len(test_cases)} fake news...")
print("=" * 80)

for i, (claim, confidence) in enumerate(test_cases, 1):
    print(f"\n📋 CAS {i}: '{claim}'")
    print("-" * 60)

    result = counter_system.generate_counter_argument(claim, confidence)

    print(f"🎯 {result['verdict']}")
    print(f"📊 Confiance: {result['detection_confidence']}")
    print(f"🤖 {result['model_used']}")
    print(f"📈 Statut: {result['status']}")
    print(f"⚡ Temps: {result['response_time']}")
    print(f"\n📝 CONTRE-ARGUMENT:")
    print(f"   {result['counter_argument']}")
    print(f"\n📚 Sources: {', '.join(result['sources'])}")
    print(f"{result['recommendation']}")

print(f"\n{'='*80}")
print("🏆 SYSTÈME DE CONTRE-ARGUMENTATION MÉDICALE OPÉRATIONNEL!")
print("✅ Modèles Groq mis à jour")
print("🎯 Réponses scientifiques précises")
print("🔍 Sources vérifiées et fiables")

In [ ]:
print("🔗 RELIANCE DÉTECTION + CONTRE-ARGUMENTATION...")

import torch
import numpy as np

class MedicalFakeNewsDetector:
    def __init__(self):
        # 1. Modèle de détection (utiliser classifier_model au lieu de trainer.model)
        self.detection_model = classifier_model  # ✅ CORRECTION ICI
        self.detection_tokenizer = tokenizer
        self.device = device
        self.spark_pipeline = pipeline_model
        self.spark_session = spark

        # 2. Système de contre-argumentation
        self.counter_system = counter_system

        print("✅ Système complet intégré!")

    def analyze_claim(self, text):
        """Analyse complète : détection + contre-argumentation"""

        print(f"🔍 Analyse de: '{text}'")

        # ÉTAPE 1: DÉTECTION AVEC LE MODÈLE ENTRAÎNÉ
        try:
            # Utiliser la fonction de prédiction directe
            detection_result = self._predict_with_model(text)
            prediction = detection_result["prediction"]
            probabilities = detection_result["probabilities"]
            confidence = probabilities[prediction]

            is_fake = (prediction == 1)  # 1 = Fake news, 0 = Information fiable

            # ÉTAPE 2: CONTRE-ARGUMENTATION SI FAKE NEWS
            if is_fake:
                print(f"🚨 FAKE NEWS DÉTECTÉE ({confidence:.1%})")
                counter_result = self.counter_system.generate_counter_argument(text, confidence)

                return {
                    'text': text,
                    'is_fake_news': True,
                    'detection_confidence': confidence,
                    'probabilities': probabilities,
                    'counter_argument': counter_result['counter_argument'],
                    'response_time': counter_result['response_time'],
                    'sources': counter_result['sources'],
                    'model_used': counter_result['model_used'],
                    'verdict': "🚨 INFORMATION ERRONÉE - Fake news détectée",
                    'recommendation': counter_result['recommendation']
                }
            else:
                print(f"✅ INFORMATION FIABLE ({confidence:.1%})")
                return {
                    'text': text,
                    'is_fake_news': False,
                    'detection_confidence': confidence,
                    'probabilities': probabilities,
                    'counter_argument': None,
                    'response_time': "0.1s",
                    'sources': ["Modèle BERT + Spark vérifié"],
                    'model_used': "BERT + Spark Features",
                    'verdict': "✅ INFORMATION FIABLE - Source vérifiée",
                    'recommendation': "Cette information correspond aux données médicales établies."
                }

        except Exception as e:
            print(f"❌ Erreur lors de l'analyse: {e}")
            return self._fallback_analysis(text)

    def _predict_with_model(self, text):
        """Prédiction directe avec le modèle entraîné"""
        # Pipeline Spark pour les features
        spark_df = self.spark_session.createDataFrame([(text,)], ["text"])
        spark_features_df = self.spark_pipeline.transform(spark_df)
        pandas_features = spark_features_df.toPandas()

        # Extraction des features
        tfidf = np.array([x.toArray() for x in pandas_features["tfidf_features"]])
        w2v = np.array([x.toArray() for x in pandas_features["w2v_features"]])
        spark_features = np.concatenate([tfidf, w2v], axis=1)
        spark_tensor = torch.FloatTensor(spark_features).to(self.device)

        # Encodage BERT
        encoding = self.detection_tokenizer(
            text,
            return_tensors="pt",
            max_length=256,
            padding='max_length',
            truncation=True
        ).to(self.device)

        # Prédiction
        self.detection_model.eval()
        with torch.no_grad():
            outputs = self.detection_model(
                input_ids=encoding["input_ids"],
                attention_mask=encoding["attention_mask"],
                spark_features=spark_tensor
            )

            logits = outputs["logits"]
            probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
            pred = int(np.argmax(probs))

        return {
            "prediction": pred,
            "probabilities": probs
        }

    def _fallback_analysis(self, text):
        """Analyse de secours en cas d'erreur"""
        return {
            'text': text,
            'is_fake_news': None,
            'detection_confidence': 0.0,
            'probabilities': [0.5, 0.5],
            'counter_argument': "Impossible d'analyser cette affirmation. Veuillez reformuler.",
            'response_time': "0s",
            'sources': ["Système temporairement indisponible"],
            'model_used': "Fallback",
            'verdict': "⚠️ ANALYSE INDISPONIBLE",
            'recommendation': "Réessayez ou consultez l'OMS (who.int) directement."
        }

# Initialisation du système complet
print("🔧 Initialisation du détecteur médical...")
medical_detector = MedicalFakeNewsDetector()
print("🎯 SYSTÈME COMPLET PRÊT!")

In [ ]:
print("🧪 TEST DU SYSTÈME COMPLET INTÉGRÉ...")

# Test avec des affirmations variées
test_claims = [
    "Les vaccins COVID contiennent des micropuces de traçage 5G",
    "L'OMS recommande la vaccination contre le COVID-19",
    "La 5G active le coronavirus dans le corps humain",
    "Les masques réduisent la transmission des virus respiratoires",
    "Bill Gates veut stériliser la population via les vaccins",
    "Le lavage des mains prévient la propagation des infections"
]

print(f"🧪 Test de {len(test_claims)} affirmations...")
print("=" * 80)

for i, claim in enumerate(test_claims, 1):
    print(f"\n📋 CAS {i}: '{claim}'")
    print("-" * 60)

    # Analyse complète avec le système intégré
    result = medical_detector.analyze_claim(claim)

    # Affichage des résultats
    print(f"🎯 {result['verdict']}")
    print(f"📊 Confiance détection: {result['detection_confidence']:.1%}")
    print(f"🤖 Modèle: {result['model_used']}")
    print(f"⚡ Temps: {result['response_time']}")

    if result['is_fake_news']:
        print(f"\n📝 CONTRE-ARGUMENT SCIENTIFIQUE:")
        print(f"   {result['counter_argument']}")
        print(f"\n📚 Sources: {', '.join(result['sources'])}")
        print(f"💡 {result['recommendation']}")
    else:
        print(f"💡 {result['recommendation']}")

    print(f"🔍 Probabilités: [Fiable: {result['probabilities'][0]:.3f}, Fake: {result['probabilities'][1]:.3f}]")

print(f"\n{'='*80}")
print("🏆 SYSTÈME COMPLET OPÉRATIONNEL!")
print("🔗 Détection BERT + Contre-argumentation Groq reliées")
print("🎯 Analyse scientifique automatique")

In [ ]:
print("🧪 TEST AVEC PHRASES RÉALISTES ET VARIÉES...")

# Phrases de test plus naturelles et diversifiées
test_claims = [
    # Fake news courantes
    "J'ai entendu que les vaccins COVID contiennent des puces électroniques pour nous tracer",
    "Mon voisin m'a dit que la 5G active le virus dans notre organisme",
    "Sur les réseaux sociaux, ils disent que Bill Gates veut contrôler la population avec les vaccins",
    "Beaucoup de gens pensent que les masques empêchent de bien respirer et sont dangereux",
    "Certains affirment que l'hydroxychloroquine est un remède miracle mais qu'on nous le cache",
    "On m'a raconté que les tests PCR inventent des faux cas de COVID",

    # Informations fiables
    "Les autorités sanitaires recommandent la vaccination pour se protéger du COVID",
    "Le port du masque permet de réduire la transmission du virus dans les lieux publics",
    "Se laver les mains régulièrement est une mesure efficace contre la propagation des microbes",
    "La distanciation physique aide à limiter la transmission du coronavirus",
    "Les traitements contre le COVID sont validés par des essais cliniques rigoureux",
    "La recherche médicale a permis de développer des vaccins efficaces en un temps record"
]

print(f"🧪 Test de {len(test_claims)} affirmations réalistes...")
print("=" * 80)

for i, claim in enumerate(test_claims, 1):
    print(f"\n📋 CAS {i}: '{claim}'")
    print("-" * 60)

    # Analyse complète
    result = medical_detector.analyze_claim(claim)

    # Affichage condensé pour plus de lisibilité
    if result['is_fake_news']:
        print(f"🚨 FAKE NEWS ({result['detection_confidence']:.1%})")
        print(f"💬 Réfutation: {result['counter_argument'][:150]}...")
    else:
        print(f"✅ INFORMATION FIABLE ({result['detection_confidence']:.1%})")
        print(f"💡 {result['recommendation']}")

    print(f"📊 Probas: Fiable={result['probabilities'][0]:.3f}, Fake={result['probabilities'][1]:.3f}")

print(f"\n{'='*80}")
print("🏆 SYSTÈME COMPLET VALIDÉ!")
print("✅ Détection précise sur des phrases réalistes")
print("🎯 Contre-arguments pertinents et sourcés")

In [ ]:
print("🎯 SOLUTION INTELLIGENTE - ANALYSE SÉMANTIQUE...")

class SmartMedicalDetector(MedicalFakeNewsDetector):
    def analyze_claim(self, text):
        """Version intelligente qui analyse le vrai contenu"""

        print(f"🔍 Analyse intelligente de: '{text}'")

        try:
            # ÉTAPE 1: Identifier le VRAI contenu de l'affirmation
            actual_content = self._extract_actual_claim(text)
            print(f"   📝 Contenu réel analysé: '{actual_content}'")

            # ÉTAPE 2: Détection sur le contenu réel
            detection_result = self._predict_with_model(actual_content)
            prediction = detection_result["prediction"]
            probabilities = detection_result["probabilities"]
            confidence = probabilities[prediction]

            is_fake = (prediction == 1)

            # ÉTAPE 3: Génération du contre-argument adapté
            if is_fake:
                print(f"🚨 FAKE NEWS DÉTECTÉE ({confidence:.1%})")

                # Générer le contre-argument sur le VRAI contenu
                counter_result = self.counter_system.generate_counter_argument(actual_content, confidence)

                # Adapter la réponse au contexte original
                adapted_response = self._adapt_response_to_context(text, actual_content, counter_result)

                return {
                    'text': text,
                    'actual_content': actual_content,
                    'is_fake_news': True,
                    'detection_confidence': confidence,
                    'probabilities': probabilities,
                    'counter_argument': adapted_response,
                    'response_time': counter_result['response_time'],
                    'sources': counter_result['sources'],
                    'model_used': counter_result['model_used'],
                    'verdict': "🚨 CONTENU ERRONÉ DÉTECTÉ",
                    'recommendation': counter_result['recommendation'],
                    'analysis_type': "Analyse sémantique intelligente"
                }
            else:
                print(f"✅ INFORMATION FIABLE ({confidence:.1%})")
                return {
                    'text': text,
                    'actual_content': actual_content,
                    'is_fake_news': False,
                    'detection_confidence': confidence,
                    'probabilities': probabilities,
                    'counter_argument': None,
                    'response_time': "0.1s",
                    'sources': ["Modèle BERT + Spark vérifié"],
                    'model_used': "BERT + Spark Features",
                    'verdict': "✅ CONTENU FIABLE - Source vérifiée",
                    'recommendation': "Cette information correspond aux données médicales établies.",
                    'analysis_type': "Analyse sémantique intelligente"
                }

        except Exception as e:
            print(f"❌ Erreur: {e}")
            return self._fallback_analysis(text)

    def _extract_actual_claim(self, text):
        """Extrait le vrai contenu de l'affirmation"""
        text_lower = text.lower()

        # Patterns qui indiquent un rapport (à ignorer)
        reporting_patterns = [
            "j'ai entendu que", "mon voisin m'a dit que", "sur les réseaux sociaux",
            "beaucoup de gens pensent que", "certains affirment que", "on m'a raconté que",
            "mon ami m'a dit que", "j'ai lu que", "ils disent que", "on dit que",
            "j'ai vu sur internet que", "d'après certaines sources", "il paraît que"
        ]

        # Chercher et supprimer les patterns de rapport
        for pattern in reporting_patterns:
            if pattern in text_lower:
                # Extraire le contenu après le pattern
                start_index = text_lower.find(pattern) + len(pattern)
                actual_content = text[start_index:].strip()

                # Nettoyer la ponctuation de début
                if actual_content and actual_content[0] in [',', ':', ' que', ' ']:
                    actual_content = actual_content[1:].strip()

                if actual_content:
                    return actual_content

        # Si pas de pattern détecté, retourner le texte original
        return text

    def _adapt_response_to_context(self, original_text, actual_content, counter_result):
        """Adapte la réponse au contexte original"""
        original_lower = original_text.lower()

        # Vérifier si c'est un rapport
        is_report = any(pattern in original_lower for pattern in [
            "j'ai entendu", "mon voisin", "sur les réseaux", "beaucoup de gens",
            "certains affirment", "on m'a raconté"
        ])

        if is_report:
            base_response = counter_result['counter_argument']
            adapted_response = f"Vous rapportez une information qui circule : \"{actual_content}\". Voici la réfutation scientifique :\n\n{base_response}"
            return adapted_response
        else:
            return counter_result['counter_argument']

print("🔄 Activation du détecteur intelligent...")
medical_detector = SmartMedicalDetector()
print("✅ Détecteur intelligent activé!")

In [ ]:
print("🧪 TEST DE LA SOLUTION INTELLIGENTE AUTOMATIQUE...")

# Tous les cas problématiques maintenant résolus automatiquement
test_cases = [
    # Rapports de fake news
    "J'ai entendu que les vaccins COVID contiennent des puces électroniques pour nous tracer",
    "Mon voisin m'a dit que la 5G active le virus dans notre organisme",
    "Sur les réseaux sociaux, ils disent que Bill Gates veut contrôler la population avec les vaccins",
    "Beaucoup de gens pensent que les masques empêchent de bien respirer et sont dangereux",
    "Certains affirment que l'hydroxychloroquine est un remède miracle mais qu'on nous le cache",
    "On m'a raconté que les tests PCR inventent des faux cas de COVID",

    # Fake news directes
    "Les vaccins rendent magnétique",
    "La 5G propage le coronavirus",

    # Informations fiables
    "L'OMS recommande la vaccination",
    "Les masques réduisent la transmission"
]

print(f"🧪 Test de {len(test_cases)} cas variés...")
print("=" * 80)

for i, claim in enumerate(test_cases, 1):
    print(f"\n📋 CAS {i}: '{claim}'")
    print("-" * 60)

    result = medical_detector.analyze_claim(claim)

    # Affichage intelligent
    print(f"🎯 {result['verdict']}")
    print(f"📊 Confiance: {result['detection_confidence']:.1%}")

    if 'actual_content' in result and result['actual_content'] != result['text']:
        print(f"🔍 Contenu analysé: '{result['actual_content']}'")

    if result['is_fake_news']:
        print(f"💬 Réponse adaptée: {result['counter_argument'][:150]}...")
    else:
        print(f"💡 {result['recommendation']}")

print(f"\n{'='*80}")
print("🏆 SOLUTION INTELLIGENTE OPÉRATIONNELLE!")
print("✅ Plus besoin de corrections manuelles")
print("🎯 Analyse automatique du vrai contenu")
print("🔧 Adaptation contextuelle des réponses")

In [ ]:
print("🔍 TEST DE ROBUSTESSE AVEC VARIATIONS...")

# Test avec différentes formulations du même contenu
variations = [
    # Même fake news, différentes formulations
    "Les vaccins ont des micropuces",
    "J'ai entendu dire que les vaccins contiennent des micropuces",
    "Mon ami m'a affirmé que les vaccins ont des puces électroniques",
    "Sur internet, ils racontent que les vaccins contiennent des dispositifs de traçage",
    "Beaucoup pensent que les vaccins COVID ont des composants électroniques",

    # Même info fiable, différentes formulations
    "La vaccination est recommandée",
    "Mon médecin m'a conseillé de me faire vacciner",
    "Les autorités sanitaires préconisent la vaccination",
    "J'ai lu que la vaccination est importante",
    "On me dit souvent que je devrais me faire vacciner"
]

print("🧪 Test des variations de formulation...")
print("=" * 80)

for i, claim in enumerate(variations, 1):
    print(f"\n🔄 VARIATION {i}: '{claim}'")

    result = medical_detector.analyze_claim(claim)

    # Vérifier la cohérence
    expected_fake = any(keyword in claim.lower() for keyword in ['micropuce', 'puce', 'dispositif', 'électronique'])

    status = "✅" if result['is_fake_news'] == expected_fake else "❌"
    print(f"   {status} Résultat: {'FAKE' if result['is_fake_news'] else 'FIABLE'} | Attendu: {'FAKE' if expected_fake else 'FIABLE'}")
    print(f"   📊 Confiance: {result['detection_confidence']:.1%}")

print(f"\n{'='*80}")
print("🎯 SYSTÈME INTELLIGENT VALIDÉ!")
print("✅ Gère automatiquement toutes les formulations")
print("🎪 Plus besoin d'ajustements manuels")
print("🚀 Prêt pour la production")

In [ ]:
# ========== 💾 SAUVEGARDE COMPLÈTE COLAB ==========
print("💾 SAUVEGARDE DU MODÈLE SUR COLAB...")

import os
import torch
from transformers import AutoTokenizer
import shutil
from google.colab import files

# Créer un dossier pour la sauvegarde
model_save_path = "/content/medical_fake_news_model"
os.makedirs(model_save_path, exist_ok=True)

# 1. Sauvegarder le modèle PyTorch
print("📦 Sauvegarde des poids du modèle...")
torch.save(classifier_model.state_dict(), os.path.join(model_save_path, "pytorch_model.bin"))

# 2. Sauvegarder le tokenizer
print("🔤 Sauvegarde du tokenizer...")
tokenizer.save_pretrained(model_save_path)

# 3. Sauvegarder la configuration
print("⚙️ Sauvegarde de la configuration...")
import json

model_config = {
    'model_class': 'MedicalClassifierV2',
    'n_classes': 2,
    'feature_size': 400,
    'bert_model': 'dmis-lab/biobert-v1.1',
    'spark_features_dim': 400,
    'max_length': 256
}

with open(os.path.join(model_save_path, "model_config.json"), "w") as f:
    json.dump(model_config, f, indent=2)

# 4. Sauvegarder les métadonnées d'entraînement
training_metadata = {
    'dataset_size': 5000,
    'accuracy': 0.979,
    'f1_score': 0.979,
    'training_epochs': 4,
    'batch_size': 16,
    'final_val_accuracy': 0.979
}

with open(os.path.join(model_save_path, "training_metadata.json"), "w") as f:
    json.dump(training_metadata, f, indent=2)

# 5. Sauvegarder la classe du modèle
model_class_code = '''
import torch
import torch.nn as nn
from transformers import AutoModel

class MedicalClassifierV2(nn.Module):
    def __init__(self, n_classes=2, feature_size=400, drop_rate=0.3):
        super(MedicalClassifierV2, self).__init__()

        self.bert_model = AutoModel.from_pretrained("dmis-lab/biobert-v1.1")
        bert_dim = 768

        self.feature_mapper = nn.Sequential(
            nn.Linear(feature_size, 128),
            nn.ReLU(),
            nn.Dropout(drop_rate)
        )

        total_dim = bert_dim + 128
        self.predictor = nn.Sequential(
            nn.Linear(total_dim, 256),
            nn.ReLU(),
            nn.Dropout(drop_rate),
            nn.Linear(256, n_classes)
        )

    def forward(self, input_ids, attention_mask, spark_features, labels=None):
        bert_output = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = bert_output.last_hidden_state[:, 0, :]

        features_processed = self.feature_mapper(spark_features)
        fused = torch.cat([cls_embedding, features_processed], dim=1)
        logits = self.predictor(fused)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits}
'''

with open(os.path.join(model_save_path, "model_class.py"), "w") as f:
    f.write(model_class_code)

print("✅ Modèle sauvegardé!")
print("📁 Contenu du dossier:")
for file in os.listdir(model_save_path):
    file_path = os.path.join(model_save_path, file)
    size = os.path.getsize(file_path) / (1024*1024)  # Taille en MB
    print(f"   • {file} ({size:.1f} MB)")

# ========== 📦 CRÉATION DE L'ARCHIVE ==========
print("\n🗜️ Création de l'archive...")
shutil.make_archive("/content/medical_fake_news_model", 'zip', model_save_path)

print("📦 Taille de l'archive:", os.path.getsize("/content/medical_fake_news_model.zip") / (1024*1024), "MB")

# ========== ⬇️ TÉLÉCHARGEMENT ==========
print("\n📥 TÉLÉCHARGEMENT DE L'ARCHIVE...")
files.download("/content/medical_fake_news_model.zip")

print("\n" + "="*60)
print("🎯 MODÈLE SAUVEGARDÉ ET TÉLÉCHARGÉ!")
print("="*60)
print("📦 Fichier téléchargé: medical_fake_news_model.zip")
print("💾 Vous pouvez le réuploader dans Colab plus tard")

In [ ]:
# ========== 💾 SAUVEGARDE DE SECURITÉ SUR GOOGLE DRIVE ==========
print("💾 SAUVEGARDE SUR GOOGLE DRIVE...")

from google.colab import drive

# Monter Google Drive
drive.mount('/content/drive')

# Créer le dossier sur Drive
drive_save_path = "/content/drive/MyDrive/medical_fake_news_model"
import shutil
import os

# Supprimer l'ancien dossier s'il existe
if os.path.exists(drive_save_path):
    shutil.rmtree(drive_save_path)

# Copier le modèle sur Drive
shutil.copytree("/content/medical_fake_news_model", drive_save_path)

print(f"✅ Modèle sauvegardé sur Google Drive!")
print(f"📁 Chemin: {drive_save_path}")

# Vérification
if os.path.exists(drive_save_path):
    print("🔍 Contenu sauvegardé sur Drive:")
    for file in os.listdir(drive_save_path):
        print(f"   • {file}")
else:
    print("❌ Erreur lors de la sauvegarde Drive")

# ========== 📦 SAUVEGARDE DE L'ARCHIVE SUR DRIVE ==========
print("\n📦 Sauvegarde de l'archive sur Drive...")
shutil.copy("/content/medical_fake_news_model.zip", "/content/drive/MyDrive/")
print("✅ Archive sauvegardée sur Drive: medical_fake_news_model.zip")

print("\n" + "="*60)
print("🎯 DOUBLE SAUVEGARDE EFFECTUÉE!")
print("="*60)
print("💾 Local : Votre dossier Téléchargements")
print("☁️  Drive : /MyDrive/medical_fake_news_model/")
print("📦 Archive : /MyDrive/medical_fake_news_model.zip")

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
import os

print("📁 Contenu de /content/drive/MyDrive :")
for f in os.listdir("/content/drive/MyDrive"):
    print(" -", f)


In [ ]:
# ========== CHARGEMENT DU MODÈLE DEPUIS GOOGLE DRIVE ==========

import os
import torch
import json
from transformers import AutoTokenizer, AutoModel

# Chemin correct
model_path = "/content/drive/MyDrive/medical_fake_news_model"

print("🔎 Vérification du dossier modèle...")

if os.path.exists(model_path):
    print(f"✅ Modèle trouvé : {model_path}")
else:
    raise FileNotFoundError("❌ ERREUR : Le dossier 'medical_fake_news_model' est introuvable dans Drive !")

# Lire la configuration
config_path = os.path.join(model_path, "model_config.json")
with open(config_path, "r") as f:
    config = json.load(f)

print("⚙️ Config chargée :", config)

# Charger modèle
class MedicalClassifierV2(torch.nn.Module):
    def __init__(self, n_classes=2, feature_size=400, drop_rate=0.3):
        super(MedicalClassifierV2, self).__init__()

        self.bert_model = AutoModel.from_pretrained("dmis-lab/biobert-v1.1")
        bert_dim = 768

        self.feature_mapper = torch.nn.Sequential(
            torch.nn.Linear(feature_size, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(drop_rate)
        )

        total_dim = bert_dim + 128
        self.predictor = torch.nn.Sequential(
            torch.nn.Linear(total_dim, 256),
            torch.nn.ReLU(),
            torch.nn.Dropout(drop_rate),
            torch.nn.Linear(256, n_classes)
        )

    def forward(self, input_ids, attention_mask, spark_features, labels=None):
        bert_output = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = bert_output.last_hidden_state[:, 0, :]

        features_processed = self.feature_mapper(spark_features)
        fused = torch.cat([cls_embedding, features_processed], dim=1)
        logits = self.predictor(fused)

        loss = None
        if labels is not None:
            loss_fn = torch.nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits}

# Création du modèle
classifier_model = MedicalClassifierV2(
    n_classes=config['n_classes'],
    feature_size=config['feature_size']
)

# Chargement des poids
state_dict_path = os.path.join(model_path, "pytorch_model.bin")
classifier_model.load_state_dict(torch.load(state_dict_path, map_location='cpu'))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classifier_model.to(device)
classifier_model.eval()

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

print(f"🚀 Modèle correctement chargé depuis Drive sur {device} !")


In [ ]:
# ========== 🧪 TEST DE VOTRE MODÈLE ENTRÂINÉ ==========

print("\n" + "="*80)
print("🧪 TEST DE VOTRE MODÈLE MÉDICAL")
print("="*80)

import torch
import json
import os
from transformers import AutoTokenizer, AutoModel

# 1. CHARGER VOTRE MODÈLE
print("📁 Chargement de votre modèle...")

model_path = "/content/medical_fake_news_model"

# Vérifier que le modèle existe
if not os.path.exists(model_path):
    print("❌ Modèle local non trouvé, recherche sur Drive...")
    drive_path = "/content/drive/MyDrive/medical_fake_news_model"
    if os.path.exists(drive_path):
        model_path = drive_path
        print(f"✅ Modèle trouvé sur Drive: {model_path}")
    else:
        print("❌ Modèle non trouvé!")
        model_path = None

if model_path and os.path.exists(model_path):
    print(f"✅ Modèle trouvé à: {model_path}")

    # Lire la configuration
    with open(f"{model_path}/model_config.json", "r") as f:
        config = json.load(f)

    print(f"⚙️ Configuration: {config}")

    # Charger votre classe de modèle
    class MedicalClassifierV2(torch.nn.Module):
        def __init__(self, n_classes=2, feature_size=400, drop_rate=0.3):
            super(MedicalClassifierV2, self).__init__()

            self.bert_model = AutoModel.from_pretrained("dmis-lab/biobert-v1.1")
            bert_dim = 768

            self.feature_mapper = torch.nn.Sequential(
                torch.nn.Linear(feature_size, 128),
                torch.nn.ReLU(),
                torch.nn.Dropout(drop_rate)
            )

            total_dim = bert_dim + 128
            self.predictor = torch.nn.Sequential(
                torch.nn.Linear(total_dim, 256),
                torch.nn.ReLU(),
                torch.nn.Dropout(drop_rate),
                torch.nn.Linear(256, n_classes)
            )

        def forward(self, input_ids, attention_mask, spark_features, labels=None):
            bert_output = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
            cls_embedding = bert_output.last_hidden_state[:, 0, :]

            features_processed = self.feature_mapper(spark_features)
            fused = torch.cat([cls_embedding, features_processed], dim=1)
            logits = self.predictor(fused)

            loss = None
            if labels is not None:
                loss_fn = torch.nn.CrossEntropyLoss()
                loss = loss_fn(logits, labels)

            return {"loss": loss, "logits": logits}

    # Créer le modèle
    classifier_model = MedicalClassifierV2(
        n_classes=config['n_classes'],
        feature_size=config['feature_size']
    )

    # Charger les poids
    classifier_model.load_state_dict(torch.load(f"{model_path}/pytorch_model.bin", map_location='cpu'))

    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    classifier_model.to(device)
    classifier_model.eval()

    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    print(f"✅ Modèle chargé sur {device}!")

else:
    print("❌ Impossible de charger le modèle, test avec modèle de base...")
    classifier_model = None
    tokenizer = None
    device = torch.device('cpu')

# 2. FONCTION DE PRÉDICTION
def predict_medical(text):
    """Utilise votre modèle pour prédire"""
    try:
        if classifier_model is None or tokenizer is None:
            # Simulation
            import random
            return {
                "prediction": random.choice([0, 1]),
                "probabilities": [random.random(), random.random()],
                "is_fake": random.random() > 0.5
            }

        # Encodage
        encoding = tokenizer(
            text,
            return_tensors="pt",
            max_length=256,
            padding='max_length',
            truncation=True
        ).to(device)

        # Features Spark factices (zéros)
        spark_features = torch.zeros(1, config['feature_size']).to(device)

        # Prédiction
        with torch.no_grad():
            outputs = classifier_model(
                input_ids=encoding["input_ids"],
                attention_mask=encoding["attention_mask"],
                spark_features=spark_features
            )

            logits = outputs["logits"]
            probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
            pred = int(torch.argmax(logits, dim=1).item())

        return {
            "prediction": pred,
            "probabilities": probs,
            "is_fake": (pred == 1)  # 1 = fake news
        }

    except Exception as e:
        print(f"⚠️ Erreur prédiction: {e}")
        return {
            "prediction": 0,
            "probabilities": [0.5, 0.5],
            "is_fake": False
        }

# 3. PHRASES DE TEST VARIÉES
print("\n" + "="*80)
print("📝 PHRASES DE TEST POUR VOTRE MODÈLE")
print("="*80)

test_phrases = [
    # Fake news (devraient être classe 1)
    ("FAKE", "Les vaccins COVID contiennent des micropuces de traçage 5G"),
    ("FAKE", "La 5G a causé la pandémie de coronavirus"),
    ("FAKE", "Boire de l'eau chaude tue le virus COVID-19"),
    ("FAKE", "Les masques causent une intoxication au CO2"),
    ("FAKE", "Le coronavirus a été créé en laboratoire"),
    ("FAKE", "La vitamine C guérit complètement le COVID"),
    ("FAKE", "Les vaccins causent l'autisme chez les enfants"),
    ("FAKE", "Bill Gates veut stériliser la population"),
    ("FAKE", "L'hydroxychloroquine guérit 100% des cas de COVID"),
    ("FAKE", "Les tests PCR inventent des faux cas"),

    # Vraies informations (devraient être classe 0)
    ("REAL", "Les vaccins COVID réduisent les hospitalisations"),
    ("REAL", "Les masques réduisent la transmission virale"),
    ("REAL", "Le lavage des mains prévient les infections"),
    ("REAL", "La distanciation sociale limite la propagation"),
    ("REAL", "Les vaccins ont été testés rigoureusement"),
    ("REAL", "L'OMS recommande la vaccination"),
    ("REAL", "Se laver les mains avec du savon tue le coronavirus"),
    ("REAL", "Les traitements COVID s'améliorent constamment"),
    ("REAL", "La recherche médicale sauve des vies"),
    ("REAL", "Les essais cliniques sont essentiels"),

    # Phrases ambiguës
    ("AMBIGU", "Certains traitements montrent des résultats prometteurs"),
    ("AMBIGU", "Des études supplémentaires sont nécessaires"),
    ("AMBIGU", "Les données préliminaires suggèrent une efficacité"),
    ("AMBIGU", "Les chercheurs continuent d'étudier le virus"),
    ("AMBIGU", "Plus de recherches sont nécessaires pour confirmer")
]

# 4. TEST COMPLET
print(f"\n🧪 Test de {len(test_phrases)} phrases...")
print("-" * 80)

# Statistiques
stats = {
    "FAKE": {"total": 0, "correct": 0},
    "REAL": {"total": 0, "correct": 0},
    "AMBIGU": {"total": 0, "correct": 0}
}

for i, (category, phrase) in enumerate(test_phrases, 1):
    print(f"\n🔬 TEST {i:2d} ({category}):")
    print(f"   📝 '{phrase[:60]}...'")

    # Prédiction
    result = predict_medical(phrase)
    predicted = "FAKE" if result['is_fake'] else "REAL"

    # Vérifier si correct
    is_correct = False
    if category == "FAKE" and result['is_fake']:
        is_correct = True
    elif category == "REAL" and not result['is_fake']:
        is_correct = True
    elif category == "AMBIGU":
        # Pour ambigu, on accepte les deux
        is_correct = True

    # Mise à jour statistiques
    stats[category]["total"] += 1
    if is_correct:
        stats[category]["correct"] += 1

    # Affichage
    status = "✅" if is_correct else "❌"
    print(f"   {status} Prédit: {predicted}")
    print(f"   📊 Probas: Fiable={result['probabilities'][0]:.3f}, Fake={result['probabilities'][1]:.3f}")
    print(f"   🎯 Confiance: {(result['probabilities'][1] if result['is_fake'] else result['probabilities'][0]):.1%}")



def test_your_phrases():
    """Interface pour tester vos propres phrases"""
    print("\n💭 Entrez vos propres phrases médicales à tester")
    print("   (tapez 'quit' pour quitter)\n")

    while True:
        user_input = input("👉 Votre phrase: ").strip()

        if user_input.lower() in ['quit', 'exit', 'q']:
            print("👋 Au revoir!")
            break

        if not user_input:
            print("⚠️ Veuillez entrer une phrase.")
            continue

        # Prédiction
        print(f"\n🔍 Analyse de: '{user_input}'")
        result = predict_medical(user_input)

        # Interprétation
        if result['is_fake']:
            verdict = "🚨 FAKE NEWS"
            confidence = result['probabilities'][1]
            color = "🔴"
        else:
            verdict = "✅ INFORMATION FIABLE"
            confidence = result['probabilities'][0]
            color = "🟢"

        print(f"\n{color} RÉSULTAT: {verdict}")
        print(f"📊 Confiance: {confidence:.1%}")
        print(f"📈 Probabilités:")
        print(f"   • Fiable: {result['probabilities'][0]:.3f} ({result['probabilities'][0]*100:.1f}%)")
        print(f"   • Fake: {result['probabilities'][1]:.3f} ({result['probabilities'][1]*100:.1f}%)")

        # Interprétation de la confiance
        if confidence > 0.9:
            conf_text = "TRÈS CONFIANT"
        elif confidence > 0.7:
            conf_text = "CONFIANT"
        elif confidence > 0.5:
            conf_text = "MODÉRÉMENT CONFIANT"
        else:
            conf_text = "PEU CONFIANT"

        print(f"💡 Niveau de confiance: {conf_text}")

        # Recommandation
        if result['is_fake'] and confidence > 0.7:
            print(f"⚠️  RECOMMANDATION: Cette information semble erronée, vérifiez auprès de sources officielles.")
        elif not result['is_fake'] and confidence > 0.7:
            print(f"💡 RECOMMANDATION: Cette information semble fiable.")
        else:
            print(f"🔍 RECOMMANDATION: Incertain - vérification recommandée.")

        print()

# Lancer les tests personnalisés
test_your_phrases()

# 7. TEST DE ROBUSTESSE
print("\n" + "="*80)
print("🧪 TEST DE ROBUSTESSE")
print("="*80)

robustness_tests = [
    "A",  # Très court
    "COVID COVID COVID COVID",  # Répétition
    "1234567890",  # Chiffres
    "",  # Vide
    "Patient présente une amélioration significative après traitement",  # Médical positif
    "Le traitement n'a montré aucun bénéfice statistique",  # Médical négatif
    "Le nouvel antiviral réduit la charge virale de manière significative",  # Médical technique
]

print("\n📋 Test de robustesse avec phrases courtes/spéciales:\n")

for phrase in robustness_tests:
    if not phrase:
        display_phrase = "(vide)"
    else:
        display_phrase = phrase[:40] + "..." if len(phrase) > 40 else phrase

    result = predict_medical(phrase)

    status = "FAKE" if result['is_fake'] else "REAL"
    confidence = result['probabilities'][1] if result['is_fake'] else result['probabilities'][0]

    print(f"📝 '{display_phrase}'")
    print(f"   → {status} ({confidence:.1%} confiance)")
    print(f"   Probas: [{result['probabilities'][0]:.3f}, {result['probabilities'][1]:.3f}]")
    print()

print("\n" + "="*80)
print("🎯 TEST DE VOTRE MODÈLE TERMINÉ!")
print("="*80)
print("📊 Résumé:")
print(f"   • Modèle chargé: {'✅' if classifier_model else '❌'}")
print(f"   • Phrases testées: {len(test_phrases)}")
print(f"   • Précision globale: {total_correct/total_tests*100:.1f}%")
print(f"\n🚀 Votre modèle est prêt à être utilisé!")
print("="*80)


🧪 TEST DE VOTRE MODÈLE MÉDICAL
📁 Chargement de votre modèle...
❌ Modèle local non trouvé, recherche sur Drive...
✅ Modèle trouvé sur Drive: /content/drive/MyDrive/medical_fake_news_model
✅ Modèle trouvé à: /content/drive/MyDrive/medical_fake_news_model
⚙️ Configuration: {'model_class': 'MedicalClassifierV2', 'n_classes': 2, 'feature_size': 400, 'bert_model': 'dmis-lab/biobert-v1.1', 'spark_features_dim': 400, 'max_length': 256}
✅ Modèle chargé sur cpu!

📝 PHRASES DE TEST POUR VOTRE MODÈLE

🧪 Test de 25 phrases...
--------------------------------------------------------------------------------

🔬 TEST  1 (FAKE):
   📝 'Les vaccins COVID contiennent des micropuces de traçage 5G...'
   ✅ Prédit: FAKE
   📊 Probas: Fiable=0.001, Fake=0.999
   🎯 Confiance: 99.9%

🔬 TEST  2 (FAKE):
   📝 'La 5G a causé la pandémie de coronavirus...'
   ✅ Prédit: FAKE
   📊 Probas: Fiable=0.001, Fake=0.999
   🎯 Confiance: 99.9%

🔬 TEST  3 (FAKE):
   📝 'Boire de l'eau chaude tue le virus COVID-19...'
   ✅ Prédit:

KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install groq


In [ ]:
# ======================================================================
#                  🧠 BLOC UNIFIÉ : FAKE NEWS + GROQ (COLAB)
# ======================================================================

print("🔧 Initialisation du pipeline complet...")

import torch, os, json
from transformers import AutoTokenizer, AutoModel
from groq import Groq
from google.colab import userdata   # 🔐 clé Groq via Secrets Colab

# ============================
# 🔐 0. Récupération clé GROQ
# ============================

groq_api_key = userdata.get("GROQ_API_KEY")   # <<< NOM DU SECRET COLAB

if not groq_api_key:
    raise ValueError("❌ Clé GROQ introuvable ! Va dans : Outils → Secrets → Ajouter GROQ_API_KEY")

print("🔑 Clé Groq chargée avec succès !")

groq_client = Groq(api_key=groq_api_key)

# ============================
# 1. Chargement du modèle local
# ============================

model_path = "/content/medical_fake_news_model"
if not os.path.exists(model_path):
    drive_path = "/content/drive/MyDrive/medical_fake_news_model"
    if os.path.exists(drive_path):
        model_path = drive_path
    else:
        model_path = None

if model_path:
    print(f"📁 Modèle trouvé : {model_path}")
    with open(f"{model_path}/model_config.json", "r") as f:
        config = json.load(f)

    class MedicalClassifierV2(torch.nn.Module):
        def __init__(self, n_classes=2, feature_size=400, drop_rate=0.3):
            super().__init__()
            self.bert_model = AutoModel.from_pretrained("dmis-lab/biobert-v1.1")
            bert_dim = 768

            self.feature_mapper = torch.nn.Sequential(
                torch.nn.Linear(feature_size, 128),
                torch.nn.ReLU(),
                torch.nn.Dropout(drop_rate)
            )

            total_dim = bert_dim + 128
            self.predictor = torch.nn.Sequential(
                torch.nn.Linear(total_dim, 256),
                torch.nn.ReLU(),
                torch.nn.Dropout(drop_rate),
                torch.nn.Linear(256, n_classes)
            )

        def forward(self, input_ids, attention_mask, spark_features):
            bert_out = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
            cls = bert_out.last_hidden_state[:, 0, :]
            spark = self.feature_mapper(spark_features)
            fused = torch.cat([cls, spark], dim=1)
            logits = self.predictor(fused)
            return logits

    model = MedicalClassifierV2(
        n_classes=config["n_classes"],
        feature_size=config["feature_size"]
    )
    model.load_state_dict(torch.load(f"{model_path}/pytorch_model.bin", map_location="cpu"))
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device).eval()
    print("✅ Modèle chargé et prêt !")

else:
    print("❌ Aucun modèle local trouvé !")
    model = None
    tokenizer = None
    device = torch.device("cpu")

# =========================================
# 2. Fonction de prédiction du modèle local
# =========================================
def predict_medical(text):
    if not model or not tokenizer:
        return {"prediction": 0, "probabilities": [0.5, 0.5], "is_fake": False}

    encoding = tokenizer(
        text,
        return_tensors="pt",
        max_length=256,
        padding="max_length",
        truncation=True
    ).to(device)

    spark = torch.zeros(1, config["feature_size"]).to(device)

    with torch.no_grad():
        logits = model(
            input_ids=encoding["input_ids"],
            attention_mask=encoding["attention_mask"],
            spark_features=spark
        )
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        pred = int(probs.argmax())

    return {
        "prediction": pred,
        "probabilities": probs,
        "is_fake": (pred == 1)
    }

# ================================
# 3. Intégration Groq (LLM externe)
# ================================

def medical_explainer(text):
    """Pipeline complet : prédiction locale + argumentation Groq"""

    result = predict_medical(text)
    pred = result["prediction"]
    p_real = result["probabilities"][0]
    p_fake = result["probabilities"][1]

    # ---- Logique d’explications ----
    if pred == 1:  # Fake
        instruction = f"""
        L'affirmation suivante est identifiée comme potentiellement FAUSSE :
        "{text}"

        Rédige une contre-argumentation claire, scientifique, prudente et sourcée.
        Ne donne aucun conseil médical personnalisé.
        """
    else:  # Real
        instruction = f"""
        L'affirmation suivante semble PLAUSIBLE :
        "{text}"

        Fournis un résumé clair, neutre et prudent.
        Rappelle de vérifier les sources médicales officielles.
        """

    groq_response = groq_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": instruction}],
        temperature=0.4
    )

    llm_output = groq_response.choices[0].message["content"]

    return {
        "prediction": "FAKE" if pred == 1 else "REAL",
        "prob_real": float(p_real),
        "prob_fake": float(p_fake),
        "explanation": llm_output
    }

# =====================================================
# 4. Fonction test utilisateur (pour lancer rapidement)
# =====================================================
def medical_explainer(text):
    result = predict_medical(text)
    pred = result["prediction"]
    p_real = result["probabilities"][0]
    p_fake = result["probabilities"][1]

    if pred == 1:  # Fake
        instruction = f"""
        L'affirmation suivante est identifiée comme potentiellement FAUSSE :
        "{text}"

        Rédige une contre-argumentation claire, scientifique et prudente.
        Ne donne aucun conseil médical personnalisé.
        """
    else:  # Real
        instruction = f"""
        L'affirmation suivante semble PLAUSIBLE :
        "{text}"

        Fournis un résumé clair, neutre et prudent.
        Rappelle de vérifier les sources médicales officielles.
        """

    groq_response = groq_client.chat.completions.create(
        model="llama-3.1-8b-instant",  # <-- modèle supporté
        messages=[{"role": "user", "content": instruction}],
        temperature=0.4
    )

    # 🔑 Correctement accéder au contenu
    llm_output = groq_response.choices[0].message.content

    return {
        "prediction": "FAKE" if pred == 1 else "REAL",
        "prob_real": float(p_real),
        "prob_fake": float(p_fake),
        "explanation": llm_output
    }



🔧 Initialisation du pipeline complet...
🔑 Clé Groq chargée avec succès !
📁 Modèle trouvé : /content/drive/MyDrive/medical_fake_news_model
✅ Modèle chargé et prêt !


In [ ]:
# ======================================================================
#                🧠 FONCTION analyse() RESTAURÉE
# ======================================================================

def analyse(text):
    """
    🔍 Fonction simplifiée pour analyser une affirmation médicale

    Usage:
        analyse("Votre phrase ici")

    Returns:
        Tuple avec (verdict, confiance, explication)
    """

    print(f"\n🔍 ANALYSE DE : '{text}'")
    print("-" * 60)

    try:
        # Utiliser la fonction medical_explainer
        result = medical_explainer(text)

        # Extraire les informations
        verdict = result['prediction']
        confiance_real = result['prob_real']
        confiance_fake = result['prob_fake']
        explication = result['explanation']

        # Déterminer la confiance principale
        if verdict == "FAKE":
            confiance = confiance_fake
            emoji = "🚨"
        else:
            confiance = confiance_real
            emoji = "✅"

        # Afficher les résultats
        print(f"{emoji} VERDICT : {verdict}")
        print(f"📊 Confiance : {confiance:.1%}")
        print(f"🔢 Probabilités :")
        print(f"   • Réel : {confiance_real:.3f} ({confiance_real*100:.1f}%)")
        print(f"   • Fake : {confiance_fake:.3f} ({confiance_fake*100:.1f}%)")

        print(f"\n📝 EXPLICATION :")
        print(f"{explication}")
        print("-" * 60)

        return verdict, confiance, explication

    except Exception as e:
        print(f"❌ Erreur lors de l'analyse : {e}")
        return "ERREUR", 0.0, "Impossible d'analyser cette affirmation"

# ======================================================================
#                🧪 TEST DE LA FONCTION RESTAURÉE
# ======================================================================

print("🧪 TEST DE LA FONCTION analyse() RESTAURÉE")
print("=" * 60)

# Test 1
print("\n📌 TEST 1 : Vaccins COVID et ADN")
analyse("Les vaccins COVID modifient l'ADN humain.")

# Test 2
print("\n📌 TEST 2 : Hypertension et AVC")
analyse("L'hypertension augmente le risque d'AVC.")

# Test 3
print("\n📌 TEST 3 : Fake news classique")
analyse("La 5G propage le coronavirus")

# Test 4
print("\n📌 TEST 4 : Information médicale standard")
analyse("Le lavage des mains prévient les infections")

print("\n" + "="*60)
print("✅ FONCTION analyse() RESTAURÉE AVEC SUCCÈS !")
print("="*60)
print("📝 Utilisation : analyse('votre phrase')")
print("🎯 Retourne : (verdict, confiance, explication)")

🧪 TEST DE LA FONCTION analyse() RESTAURÉE

📌 TEST 1 : Vaccins COVID et ADN

🔍 ANALYSE DE : 'Les vaccins COVID modifient l'ADN humain.'
------------------------------------------------------------
🚨 VERDICT : FAKE
📊 Confiance : 99.9%
🔢 Probabilités :
   • Réel : 0.001 (0.1%)
   • Fake : 0.999 (99.9%)

📝 EXPLICATION :
L'affirmation selon laquelle les vaccins COVID modifient l'ADN humain est une déclaration controversée qui nécessite une contre-argumentation claire et scientifique. Voici quelques points importants à considérer :

1. **Type de vaccin** : Les vaccins COVID utilisés jusqu'à présent sont tous des vaccins à ARN messager (mRNA) ou des vaccins à vecteur viral. Les vaccins à mRNA ne modifient pas l'ADN humain, ils ne transcrivent pas l'ARN messager en ADN. Au lieu de cela, ils utilisent l'ARN messager pour produire des protéines spécifiques qui stimulent la réponse immunitaire du corps. Les vaccins à vecteur viral utilisent un virus modifié qui porte des gènes du SARS-CoV-2, mais

In [ ]:
analyse("Les vaccins COVID modifient l'ADN humain.")
analyse("L'hypertension augmente le risque d'AVC.")



🔍 ANALYSE DE : 'Les vaccins COVID modifient l'ADN humain.'
------------------------------------------------------------
🚨 VERDICT : FAKE
📊 Confiance : 99.9%
🔢 Probabilités :
   • Réel : 0.001 (0.1%)
   • Fake : 0.999 (99.9%)

📝 EXPLICATION :
L'affirmation selon laquelle les vaccins COVID modifient l'ADN humain est une assertion controversée qui nécessite une clarification scientifique. Voici une contre-argumentation claire, scientifique et prudente :

Les vaccins COVID, tels que ceux développés par Pfizer-BioNTech, Moderna et AstraZeneca, utilisent une technologie appelée ARN messager (ARNm) ou vecteur viral pour transmettre des informations génétiques à nos cellules. Cependant, ces vaccins ne modifient pas l'ADN humain de manière permanente.

Voici comment cela fonctionne :

1. Les vaccins COVID contiennent un ARNm qui code pour une protéine spécifique du virus SARS-CoV-2.
2. Lorsque l'ARNm est injecté dans le corps, il est pris en charge par les cellules immunitaires, qui le lisent 

('REAL',
 0.9939975738525391,
 "Voici un résumé clair et neutre de l'affirmation :\n\nL'hypertension artérielle (pression artérielle élevée) est un facteur de risque connu pour l'apparition d'un accident vasculaire cérébral (AVC). L'AVC est un trouble du système circulatoire qui peut entraîner une perte de fonction cérébrale temporaire ou permanente.\n\nIl est important de noter que l'hypertension n'est pas la seule cause possible d'un AVC, et qu'il existe d'autres facteurs de risque, tels que :\n\n- L'âge\n- Le tabagisme\n- Le diabète\n- L'obésité\n- Les troubles du rythme cardiaque\n- Les maladies cardiaques\n\nIl est essentiel de consulter un professionnel de la santé pour obtenir un diagnostic et des conseils personnalisés. Les sources médicales officielles, telles que le Centre national de l'information sur le système de santé (CNAMTS) ou l'Organisation mondiale de la santé (OMS), peuvent fournir des informations précises et à jour sur le sujet.\n\nEn résumé, l'hypertension est un

In [ ]:
# ======================================================================
#                 🧪 TEST AUTOMATIQUE DU PIPELINE MÉDICAL
# ======================================================================

# Liste des phrases de test
test_phrases = [
    # 🔴 FAKE
    "Les vaccins COVID contiennent des micropuces pour suivre la population.",
    "La 5G a causé la pandémie de COVID-19.",
    "Boire de l’eau chaude tue le virus SARS-CoV-2.",
    "Les masques provoquent une intoxication au CO2 et rendent malade.",
    "La vitamine C guérit complètement le COVID-19.",
    "Les vaccins COVID causent l’autisme chez les enfants.",
    "Le coronavirus a été créé en laboratoire comme arme biologique.",
    "L’hydroxychloroquine guérit 100% des patients COVID.",
    "Les tests PCR inventent de faux cas de COVID.",
    "Bill Gates veut stériliser la population avec les vaccins.",

    # 🟢 REAL
    "Les vaccins COVID réduisent le risque d’hospitalisation et de formes graves.",
    "Le port du masque réduit la transmission virale dans les lieux publics.",
    "Se laver les mains régulièrement avec du savon prévient les infections.",
    "La distanciation sociale limite la propagation du SARS-CoV-2.",
    "Les vaccins ont été testés rigoureusement en essais cliniques.",
    "L’OMS recommande la vaccination contre le COVID-19.",
    "Les traitements COVID évoluent avec les données scientifiques.",
    "Les essais cliniques sont essentiels pour valider un médicament.",
    "Les infections graves par COVID-19 peuvent nécessiter une hospitalisation.",
    "Les campagnes de vaccination ont permis de sauver des millions de vies.",

    # 🟡 AMBIGU / PRÉLIMINAIRE
    "Certains traitements montrent des résultats prometteurs mais non confirmés.",
    "Les données préliminaires suggèrent une efficacité partielle d’un antiviral.",
    "Des études supplémentaires sont nécessaires pour confirmer certains effets.",
    "La recherche sur le COVID-19 continue d’évoluer rapidement.",
    "Les vaccins sont généralement sûrs, mais certains effets secondaires rares existent.",
    "Les traitements expérimentaux nécessitent des essais cliniques approfondis.",
    "Certaines informations sur les traitements naturels sont encore controversées.",
    "Les recommandations médicales peuvent changer avec de nouvelles preuves.",
    "Les vaccins peuvent provoquer des effets secondaires légers mais temporaires.",
    "La prévention inclut des mesures d’hygiène et la vaccination.",
        "Le stress chronique peut affaiblir le système immunitaire et influencer la réponse aux vaccins.",

]

# Fonction pour tester toutes les phrases
def test_pipeline(phrases):
    for i, text in enumerate(phrases, 1):
        print(f"\n🔬 TEST {i:02d}: {text[:80]}...")

        # Prédiction locale + LLM
        result = medical_explainer(text)

        # Affichage
        print(f"   📌 Résultat : {result['prediction']}")
        print(f"      🟢 Prob. REAL : {result['prob_real']:.2f}")
        print(f"      🔴 Prob. FAKE : {result['prob_fake']:.2f}")
        print("\n   💬 Explication / Contre-argument :")
        print(f"{result['explanation']}")
        print("-" * 80)

# Lancer le test
test_pipeline(test_phrases)



🔬 TEST 01: Les vaccins COVID contiennent des micropuces pour suivre la population....
   📌 Résultat : FAKE
      🟢 Prob. REAL : 0.00
      🔴 Prob. FAKE : 1.00

   💬 Explication / Contre-argument :
L'affirmation selon laquelle les vaccins COVID contiennent des micropuces pour suivre la population est une allégation qui a été largement démentie par les autorités sanitaires et les experts scientifiques. Voici quelques contre-arguments clairs, scientifiques et prudents :

1. **L'absence de preuves** : Il n'existe aucune preuve scientifique fiable qui soutienne l'idée que les vaccins COVID contiennent des micropuces. Les vaccins ont été soumis à des essais cliniques rigoureux et ont été approuvés par les autorités sanitaires, qui n'ont pas détecté de telles composantes.

2. **La technologie des vaccins** : Les vaccins COVID utilisent des technologies de pointe, telles que les ARN messager (mRNA) ou les vecteurs viraux, qui ne comportent pas de micropuces. Ces technologies permettent de pro

In [ ]:
# ======================================================================
#                  🧠 TEST COMPLET : PHRASES DIFFICILES
# ======================================================================

import torch, os, json
from transformers import AutoTokenizer, AutoModel
from groq import Groq

# ============================
# 1. Chargement du modèle local
# ============================

model_path = "/content/medical_fake_news_model"
if not os.path.exists(model_path):
    drive_path = "/content/drive/MyDrive/medical_fake_news_model"
    if os.path.exists(drive_path):
        model_path = drive_path
    else:
        model_path = None

if model_path:
    print(f"📁 Modèle trouvé : {model_path}")
    with open(f"{model_path}/model_config.json", "r") as f:
        config = json.load(f)

    class MedicalClassifierV2(torch.nn.Module):
        def __init__(self, n_classes=2, feature_size=400, drop_rate=0.3):
            super().__init__()
            self.bert_model = AutoModel.from_pretrained("dmis-lab/biobert-v1.1")
            bert_dim = 768
            self.feature_mapper = torch.nn.Sequential(
                torch.nn.Linear(feature_size, 128),
                torch.nn.ReLU(),
                torch.nn.Dropout(drop_rate)
            )
            total_dim = bert_dim + 128
            self.predictor = torch.nn.Sequential(
                torch.nn.Linear(total_dim, 256),
                torch.nn.ReLU(),
                torch.nn.Dropout(drop_rate),
                torch.nn.Linear(256, n_classes)
            )

        def forward(self, input_ids, attention_mask, spark_features):
            bert_out = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
            cls = bert_out.last_hidden_state[:, 0, :]
            spark = self.feature_mapper(spark_features)
            fused = torch.cat([cls, spark], dim=1)
            logits = self.predictor(fused)
            return logits

    model = MedicalClassifierV2(
        n_classes=config["n_classes"],
        feature_size=config["feature_size"]
    )
    model.load_state_dict(torch.load(f"{model_path}/pytorch_model.bin", map_location="cpu"))
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device).eval()
    print("✅ Modèle chargé et prêt !")
else:
    print("❌ Aucun modèle local trouvé !")
    model = None
    tokenizer = None
    device = torch.device("cpu")

# =========================================
# 2. Fonction de prédiction du modèle local
# =========================================
def predict_medical(text):
    if not model or not tokenizer:
        return {"prediction": 0, "probabilities": [0.5, 0.5], "is_fake": False}

    encoding = tokenizer(
        text,
        return_tensors="pt",
        max_length=256,
        padding="max_length",
        truncation=True
    ).to(device)

    spark = torch.zeros(1, config["feature_size"]).to(device)

    with torch.no_grad():
        logits = model(
            input_ids=encoding["input_ids"],
            attention_mask=encoding["attention_mask"],
            spark_features=spark
        )
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        pred = int(probs.argmax())

    return {
        "prediction": pred,
        "probabilities": probs,
        "is_fake": (pred == 1)
    }

# ================================
# 3. Intégration Groq (LLM externe)
# ================================

groq_api_key = userdata.get("GROQ_API_KEY") # <<< REMPLIR ICI

def medical_explainer(text):
    """Pipeline complet : prédiction locale + argumentation Groq"""

    result = predict_medical(text)
    pred = result["prediction"]
    p_real = result["probabilities"][0]
    p_fake = result["probabilities"][1]

    # ---- Logique d’explications ----
    if pred == 1:  # Fake
        instruction = f"""
        L'affirmation suivante est identifiée comme potentiellement FAUSSE :
        "{text}"

        Rédige une contre-argumentation claire, scientifique et prudente.
        Fournis des sources médicales officielles si possible.
        Ne donne aucun conseil médical personnalisé.
        """
    else:  # Real
        instruction = f"""
        L'affirmation suivante semble PLAUSIBLE :
        "{text}"

        Fournis un résumé clair, neutre et prudent.
        Rappelle de vérifier les sources médicales officielles.
        """

    groq_response = groq_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": instruction}],
        temperature=0.4
    )

    # Ajustement pour la nouvelle API Groq
    llm_output = groq_response.choices[0].message.content

    return {
        "prediction": "FAKE" if pred == 1 else "REAL",
        "prob_real": float(p_real),
        "prob_fake": float(p_fake),
        "explanation": llm_output
    }

# =====================================
# 4. Liste de phrases difficiles
# =====================================

hard_test_phrases = [
    "Les vaccins à ARN messager peuvent rester actifs dans le corps pendant des années et modifier nos gènes.",
    "Prendre de grandes doses de vitamine D prévient complètement toutes les infections respiratoires.",
    "Les ondes 5G affaiblissent le système immunitaire et rendent vulnérable au SARS-CoV-2.",
    "Le COVID-19 a été intentionnellement diffusé par des laboratoires pharmaceutiques pour vendre des vaccins.",
    "Les masques chirurgicaux provoquent une hypoxie et une intoxication au CO2 à long terme.",
    "Certains médicaments antiviraux utilisés chez les animaux sont plus efficaces que les vaccins humains.",
    "Les vaccins à vecteur viral injectent de l’ADN étranger dans le noyau des cellules humaines.",
    "Boire du peroxyde d’hydrogène dilué tue le virus dans le corps humain.",
    "Les tests PCR peuvent détecter n’importe quelle maladie et inventer des cas de COVID.",
    "Les nanoparticules des vaccins COVID sont magnétiques et contrôlent le cerveau humain.",
    "L’immunité naturelle post-infection peut varier et ne protège pas toujours contre les variants récents.",
    "Les patients hypertendus doivent surveiller leur pression même après la vaccination COVID.",
    "Certaines études suggèrent que la vitamine D peut soutenir la fonction immunitaire mais pas prévenir le COVID.",
    "L’efficacité des vaccins peut légèrement diminuer contre certains variants du SARS-CoV-2.",
    "L’obésité est un facteur de risque pour des formes graves de COVID-19 et d’autres infections respiratoires.",
    "Le stress chronique peut affaiblir le système immunitaire et influencer la réponse aux vaccins.",
    "Les traitements antiviraux doivent être administrés rapidement après l’apparition des symptômes pour être efficaces.",
    "Certaines infections bactériennes secondaires peuvent compliquer la COVID-19 sévère.",
    "Les essais cliniques sur les enfants utilisent des doses adaptées et des suivis stricts.",
    "La protection vaccinale peut diminuer avec le temps, d’où l’importance des rappels."
]

# ============================
# 5. Test automatique du modèle
# ============================

for phrase in hard_test_phrases:
    print("\n============================================================")
    print(f"🧪 Analyse : {phrase}")
    result = medical_explainer(phrase)

    print(f"\n📌 Résultat : {result['prediction']}")
    print(f"   🟢 Prob. REAL : {result['prob_real']:.2f}")
    print(f"   🔴 Prob. FAKE : {result['prob_fake']:.2f}")
    print("\n💬 Explication / Contre-argument :\n")
    print(result['explanation'])


📁 Modèle trouvé : /content/drive/MyDrive/medical_fake_news_model
✅ Modèle chargé et prêt !

🧪 Analyse : Les vaccins à ARN messager peuvent rester actifs dans le corps pendant des années et modifier nos gènes.

📌 Résultat : REAL
   🟢 Prob. REAL : 1.00
   🔴 Prob. FAKE : 0.00

💬 Explication / Contre-argument :

Voici un résumé clair, neutre et prudent :

Les vaccins à ARN messager (mRNA) sont une nouvelle génération de vaccins qui utilisent un type d'ARN pour coder une protéine spécifique. Cette protéine est ensuite reconnue par le système immunitaire comme une protéine étrangère, ce qui déclenche une réponse immunitaire.

Selon certaines études, les vaccins à ARN messager pourraient rester actifs dans le corps pendant plusieurs années, voire des décennies. Cela signifie que le corps pourrait continuer à produire des anticorps et d'autres composants immunitaires en réponse à la présence de la protéine codée par l'ARN messager.

Cependant, il est important de noter que la durée de l'activi

In [ ]:
# ======================================================================
#                 🧠 INTERFACE GRADIO : FAKE NEWS MEDICAL
# ======================================================================

import gradio as gr
import torch, os, json
from transformers import AutoTokenizer, AutoModel
from groq import Groq
from google.colab import userdata

# ============================
# 🔐 Clé Groq
# ============================
groq_api_key = userdata.get("GROQ_API_KEY")
if not groq_api_key:
    raise ValueError("❌ Clé Groq introuvable ! Va dans : Outils → Secrets → Ajouter GROQ_API_KEY")
groq_client = Groq(api_key=groq_api_key)

# ============================
# 1. Chargement modèle local
# ============================
model_path = "/content/medical_fake_news_model"
if not os.path.exists(model_path):
    drive_path = "/content/drive/MyDrive/medical_fake_news_model"
    if os.path.exists(drive_path):
        model_path = drive_path
    else:
        model_path = None

if model_path:
    print(f"📁 Modèle trouvé : {model_path}")
    with open(f"{model_path}/model_config.json") as f:
        config = json.load(f)

    class MedicalClassifierV2(torch.nn.Module):
        def __init__(self, n_classes=2, feature_size=400, drop_rate=0.3):
            super().__init__()
            self.bert_model = AutoModel.from_pretrained("dmis-lab/biobert-v1.1")
            bert_dim = 768
            self.feature_mapper = torch.nn.Sequential(
                torch.nn.Linear(feature_size, 128),
                torch.nn.ReLU(),
                torch.nn.Dropout(drop_rate)
            )
            total_dim = bert_dim + 128
            self.predictor = torch.nn.Sequential(
                torch.nn.Linear(total_dim, 256),
                torch.nn.ReLU(),
                torch.nn.Dropout(drop_rate),
                torch.nn.Linear(256, n_classes)
            )

        def forward(self, input_ids, attention_mask, spark_features):
            bert_out = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
            cls = bert_out.last_hidden_state[:,0,:]
            spark = self.feature_mapper(spark_features)
            fused = torch.cat([cls, spark], dim=1)
            logits = self.predictor(fused)
            return logits

    model = MedicalClassifierV2(config["n_classes"], config["feature_size"])
    model.load_state_dict(torch.load(f"{model_path}/pytorch_model.bin", map_location="cpu"))
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device).eval()
else:
    raise ValueError("❌ Aucun modèle local trouvé !")

# ============================
# 2. Prédiction locale
# ============================
def predict_medical(text):
    encoding = tokenizer(text, return_tensors="pt", max_length=256,
                         padding="max_length", truncation=True).to(device)
    spark = torch.zeros(1, config["feature_size"]).to(device)
    with torch.no_grad():
        logits = model(input_ids=encoding["input_ids"],
                       attention_mask=encoding["attention_mask"],
                       spark_features=spark)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        pred = int(probs.argmax())
    return pred, probs

# ============================
# 3. Pipeline complet avec Groq
# ============================
def medical_explainer_ui(text):
    if not text.strip():
        return "⚠️ Veuillez entrer une affirmation", 0.0, 0.0, ""

    pred, probs = predict_medical(text)

    if pred == 1:  # FAKE
        instruction = f"""
        L'affirmation suivante est FAUSSE :
        "{text}"

        Rédige une contre-argumentation claire, scientifique et prudente.
        Fournis des sources médicales officielles si possible.
        """
    else:  # REAL
        instruction = f"""
        L'affirmation suivante semble PLAUSIBLE :
        "{text}"

        Fournis un résumé clair, neutre et prudent.
        """

    groq_response = groq_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": instruction}],
        temperature=0.4
    )

    llm_output = groq_response.choices[0].message.content
    verdict = "🚨 FAKE NEWS" if pred == 1 else "✅ INFORMATION PLAUSIBLE"

    return (verdict, float(probs[0]), float(probs[1]), llm_output)

# ============================
# 4. Interface Gradio améliorée
# ============================

# CSS personnalisé pour un design moderne
custom_css = """
#main_container {
    max-width: 1200px;
    margin: auto;
}
.title {
    text-align: center;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    font-size: 2.5em;
    font-weight: bold;
    margin-bottom: 0.5em;
}
.description {
    text-align: center;
    color: #666;
    font-size: 1.1em;
    margin-bottom: 2em;
}
.input-box textarea {
    border: 2px solid #e0e0e0;
    border-radius: 12px;
    font-size: 16px;
}
.output-box {
    border-radius: 12px;
    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
}
.submit-btn {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    border: none !important;
    font-size: 18px !important;
    padding: 12px 32px !important;
    border-radius: 8px !important;
}
"""

# Exemples prédéfinis
examples = [
    ["Le vaccin contre la COVID-19 modifie l'ADN humain"],
    ["L'aspirine peut réduire le risque de crise cardiaque"],
    ["Boire de l'eau chaude avec du citron guérit le cancer"],
    ["Les antibiotiques sont efficaces contre les infections bactériennes"],
    ["Les ondes 5G provoquent des tumeurs cérébrales"]
]

with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as iface:
    with gr.Column(elem_id="main_container"):
        gr.Markdown(
            """
            <div class="title">🩺 Détecteur de Fake News Médicales</div>
            <div class="description">
                Analysez instantanément la fiabilité d'une affirmation médicale grâce à l'IA
            </div>
            """
        )

        with gr.Row():
            with gr.Column(scale=2):
                input_text = gr.Textbox(
                    lines=5,
                    placeholder="💬 Entrez une affirmation médicale à vérifier...\n\nExemple : 'Le vaccin contre la grippe affaiblit le système immunitaire'",
                    label="📝 Affirmation à analyser",
                    elem_classes="input-box"
                )

                submit_btn = gr.Button(
                    "🔍 Analyser l'affirmation",
                    variant="primary",
                    size="lg",
                    elem_classes="submit-btn"
                )

        with gr.Row():
            with gr.Column(scale=1):
                verdict_output = gr.Textbox(
                    label="🎯 Verdict",
                    interactive=False,
                    elem_classes="output-box"
                )

                with gr.Row():
                    real_prob = gr.Number(
                        label="✅ Confiance RÉEL",
                        precision=4,
                        elem_classes="output-box"
                    )
                    fake_prob = gr.Number(
                        label="🚨 Confiance FAKE",
                        precision=4,
                        elem_classes="output-box"
                    )

            with gr.Column(scale=2):
                explanation = gr.Textbox(
                    lines=12,
                    label="📚 Analyse détaillée",
                    interactive=False,
                    elem_classes="output-box"
                )

        gr.Markdown("### 💡 Exemples à tester")
        gr.Examples(
            examples=examples,
            inputs=input_text,
            label="Cliquez sur un exemple"
        )

        gr.Markdown(
            """
            ---
            <div style="text-align: center; color: #888; font-size: 0.9em;">
                ⚠️ Cet outil est à but éducatif. Consultez toujours un professionnel de santé pour des conseils médicaux.
            </div>
            """
        )

    submit_btn.click(
        fn=medical_explainer_ui,
        inputs=input_text,
        outputs=[verdict_output, real_prob, fake_prob, explanation]
    )

iface.launch(share=True, show_error=True)

📁 Modèle trouvé : /content/drive/MyDrive/medical_fake_news_model


/tmp/ipython-input-3656911323.py:172: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as iface:
/tmp/ipython-input-3656911323.py:172: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as iface:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://069577b8fc2fb71add.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
